In [ ]:
import numpy as np
import pandas as pd
from tqdm import tqdm
import seaborn as sns
from scipy import stats
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
import matplotlib.dates as mdates
from matplotlib.colors import ListedColormap
import pickle
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, roc_curve
import lightgbm as lgb
from sklearn.model_selection import KFold
from imblearn.under_sampling import RandomUnderSampler
import statsmodels.formula.api as smf
import statsmodels.stats.sandwich_covariance as sw
import joblib
import joypy
from matplotlib.lines import Line2D
import shutil

import scipy.stats as st
# import mord
from scipy import stats 
from scipy.stats import t
from datetime import datetime, date, timedelta
from dateutil.relativedelta import relativedelta
# from tabulate import tabulate
from pandas.api.types import CategoricalDtype
from sklearn.model_selection import train_test_split 
from sklearn.linear_model import LinearRegression 
from statsmodels.formula.api import ols
from matplotlib.patches import Patch

import statsmodels.formula.api as smf
import statsmodels.stats.sandwich_covariance as sw

In [ ]:

dict_label = {
    "age": "Age",
    "gender": "Gender",
    "watch_all_per_day": "Daily Watch Time (s)",
    "watch_all_per_day_min": "Watch Time (min)",
    "coverage_per_2weeks": "Days Watched",
    "category_count_unique": "Category Counts",
    "session_all_per_day": "Watch Sessions",
    "midnight_wt": "Watch Time (s)",
    "midnight_wt_min": "Watch Time (min)",
    "morning_wt_min": "Watch Time (min)",
    "noon_wt_min": "Watch Time (min)",
    "afternoon_wt_min": "Watch Time (min)",
    "evening_wt_min": "Watch Time (min)",
      "pid_SHUA_all_per_day": "Exposed Videos",  # Number of 'SHUA' videos watched per day
    "pid_watch_all_per_day": "Watched Videos",  # Total number of videos watched per day
    'category_normalized_videos': 'Category per Video',
    'category_normalized_watch_time': 'Category per Watch Time',
    "hour_0_wt": "Watch Time (0-1 AM)",
    "hour_1_wt": "Watch Time (1-2 AM)",
    "hour_2_wt": "Watch Time (2-3 AM)",
    "hour_3_wt": "Watch Time (3-4 AM)",
    "hour_4_wt": "Watch Time (4-5 AM)",
    "hour_5_wt": "Watch Time (5-6 AM)",
    "hour_6_wt": "Watch Time (6-7 AM)",
    "hour_7_wt": "Watch Time (7-8 AM)",
    "hour_8_wt": "Watch Time (8-9 AM)",
    "hour_9_wt": "Watch Time (9-10 AM)",
    "hour_10_wt": "Watch Time (10-11 AM)",
    "hour_11_wt": "Watch Time (11-12 PM)",
    "hour_12_wt": "Watch Time (12-1 PM)",
    "hour_13_wt": "Watch Time (1-2 PM)",
    "hour_14_wt": "Watch Time (2-3 PM)",
    "hour_15_wt": "Watch Time (3-4 PM)",
    "hour_16_wt": "Watch Time (4-5 PM)",
    "hour_17_wt": "Watch Time (5-6 PM)",
    "hour_18_wt": "Watch Time (6-7 PM)",
    "hour_19_wt": "Watch Time (7-8 PM)",
    "hour_20_wt": "Watch Time (8-9 PM)",
    "hour_21_wt": "Watch Time (9-10 PM)",
    "hour_22_wt": "Watch Time (10-11 PM)",
    "hour_23_wt": "Watch Time (11 PM-0 AM)",
    "watch_all_per_week": "Weekly Watch Time (Seconds)",  # Total watch time per week (in seconds)
    "watch_all_per_2weeks": "Biweekly Watch Time (Seconds)",  # Total watch time over 2 weeks (in seconds)
    "watch_all_per_session": "Watch Time per Session (Seconds)",  # Average watch time per session (in seconds)
    "duration_per_day": "Daily Video Duration (Seconds)",  # Average video duration watched per day (in seconds)
    "longvideo_count_per_day": "Daily Long Video Count",  # Number of long videos (duration > 15 mins) watched per day
    "shortvideo_count_per_day": "Daily Short Video Count",  # Number of short videos (duration <= 15 mins) watched per day
    "coverage_per_week": "Days Watched per Week",  # Number of days user watched content in a week
    "root_category_count_unique": "Unique Root Video Categories Watched",  # Number of unique root-level categories watched
    "pid_0s_per_day_nunique": "Unique 'PID 0s' Watched per Day",  # Number of unique 'PID 0' videos watched per day
    "fre_city_level": "User's City Level"  # User's city level (e.g., Tier 1, Tier 2)
}

feature_xgboost = [
"age", 'gender', 'watch_all_per_day_min', 'coverage_per_2weeks',
    "category_count_unique", 
    'session_all_per_day', 'midnight_wt_min',
    'morning_wt_min', 
    'noon_wt_min', 
    'afternoon_wt_min', 
    'evening_wt_min',
    'pid_SHUA_all_per_day', 
    'pid_watch_all_per_day',

]


In [ ]:
df_combined_all = pd.read_csv('dataset.csv', encoding="utf_8_sig")


In [ ]:
df_survey_users = pd.read_csv('survey_firstmonth_data.csv', encoding="utf_8_sig")

In [ ]:
## preds_3labelgood
label_mapping = {
    0: 'Non-Addicted',
    1: 'Mildly Addicted',
    2: 'Severely Addicted'
}

# Use map() to create a new column 'preds_3_label_criteria_label' with the descriptive labels
df_combined_all['preds_3_label_criteria_label'] = df_combined_all['preds_3_label_criteria'].map(label_mapping)


label_mapping = {
    0: 'Non-Addicted',
    1: 'Addicted',
}

# Use map() to create a new column 'preds_3_label_criteria_label' with the descriptive labels
df_combined_all['preds_soft_label'] = df_combined_all['preds_soft'].map(label_mapping)


In [ ]:
df_survey_users['watch_all_per_day_min'] = df_survey_users['watch_all_per_day']//60
df_survey_users['midnight_wt_min'] = df_survey_users['midnight_wt']//60
df_survey_users['morning_wt_min'] = df_survey_users['morning_wt']//60
df_survey_users['noon_wt_min'] = df_survey_users['noon_wt']//60
df_survey_users['afternoon_wt_min'] = df_survey_users['afternoon_wt']//60
df_survey_users['evening_wt_min'] = df_survey_users['evening_wt']//60

df_survey_users[['watch_all_per_day_min','watch_all_per_day','noon_wt_min']].max()

df_combined_all['watch_all_per_day_min'] = df_combined_all['watch_all_per_day']//60
df_combined_all['midnight_wt_min'] = df_combined_all['midnight_wt']//60
df_combined_all['morning_wt_min'] = df_combined_all['morning_wt']//60
df_combined_all['noon_wt_min'] = df_combined_all['noon_wt']//60
df_combined_all['afternoon_wt_min'] = df_combined_all['afternoon_wt']//60
df_combined_all['evening_wt_min'] = df_combined_all['evening_wt']//60

df_combined_all[['watch_all_per_day_min','watch_all_per_day','noon_wt_min']].max()

In [ ]:
age_orders = ["0-12", "13-17", "18-24", "25-34", "35-44", "45-54", "55-64", "65+"]

# Function to assign age segments based on age ranges
def separate_age(age, age_orders):
    for age_order in age_orders[:-1]:
        start = int(age_order.split('-')[0])
        end = int(age_order.split('-')[1])
        if age <= end and age >= start:
            return age_order
    return age_orders[-1]

# Apply age segmentation
df_combined_all['age_segment'] = df_combined_all['age'].apply(lambda x: separate_age(x, age_orders))


In [ ]:
df_combined_month0 = df_combined_all[df_combined_all['month']==0]

In [ ]:
### Plot 1: preds_soft (Addicted vs Non-Addicted) based on proportions across all age groups
plt.rcParams.update({'font.weight': 'normal','font.size': '20', 'axes.labelsize': 'x-large', 'font.sans-serif': 'Arial'})

# Group by age_segment and preds_soft to calculate the counts for each group
df_counts_soft = df_combined_month0.groupby(['age_segment', 'preds_soft']).size().reset_index(name='count')

# Calculate the total count for each preds_soft group (Addicted and Non-Addicted)
df_counts_soft['total_soft'] = df_counts_soft.groupby('preds_soft')['count'].transform('sum')

# Calculate the proportion for each age_segment within the entire Addicted/Non-Addicted group
# df_counts_soft['proportion'] = df_counts_soft['count'] / df_counts_soft['total_soft']
df_counts_soft['proportion'] = df_counts_soft['count'] / df_counts_soft['total_soft']

# Custom palette for hue = 'preds_soft' (Addicted vs Non-Addicted)
blue_palette = sns.color_palette("Blues", 6)
custom_palette_soft = {0: blue_palette[1], 1: blue_palette[4]}  # 0 = Non-Addicted, 1 = Addicted

# Create the bar plot for preds_soft (Addicted vs Non-Addicted) using the custom palette
g_soft = sns.catplot(
    data=df_counts_soft,  
    x="age_segment", y='proportion', hue='preds_soft', 
    order=age_orders, hue_order=[1, 0],
    palette=custom_palette_soft,  
    height=4.5, aspect=2.15, kind="bar", legend=False
)

# Set labels and formatting
plt.xlabel("", fontsize="20")
plt.ylabel("Proportion", fontsize="22")
plt.xticks(fontsize="25")
plt.yticks(fontsize="23")

# Get handles and labels from the plot
handles, labels = g_soft.ax.get_legend_handles_labels()
g_soft.ax.spines[['right', 'top']].set_visible(True)

# Set the custom labels for Addicted and Non-Addicted
plt.legend(handles=handles, labels=['Addicted', 'Non-Addicted'], fontsize=22, loc="upper right")

# Adjust layout and save the plot
plt.tight_layout()
plt.savefig('./addiction1_xgboost_20241201revision_output/figs/age_soft_proportion.pdf', format='pdf', dpi=1000, bbox_inches='tight')
plt.show()


min_age = df_combined_month0['age'].min()
max_age = df_combined_month0['age'].max()
# Plot density with filled areas for each group
figure, ax = plt.subplots(figsize=(8, 4))
sns.kdeplot(df_combined_month0[df_combined_month0['preds_soft'] == 1]['age'], shade=True, color=custom_palette_soft[1], zorder=2,label='Addicted', alpha=0.5)
sns.kdeplot(df_combined_month0[df_combined_month0['preds_soft'] == 0]['age'], shade=True, color=custom_palette_soft[0], zorder=1,label='Non-Addicted', alpha=0.7)
plt.xlabel("Age", fontsize="22")
plt.ylabel("Density", fontsize="21")
plt.xlim(0, 90)  # Extend to cover ages below 11 and above 77
plt.xticks(range(0,90,6), fontsize=22)
plt.yticks(fontsize=22)
plt.legend(handles=handles, labels=['Addicted', 'Non-Addicted'], fontsize=22, loc="upper right")

# plt.grid(color='gray', linestyle='--', linewidth=0.5, alpha=0.4)
plt.show()



### Plot 2: preds_3_label_criteria (Severely Addicted, Mildly Addicted, Non-Addicted) based on proportions across all age groups

# Group by age_segment and preds_3_label_criteriaGood to calculate the counts for each group
df_counts_3labels = df_combined_month0.groupby(['age_segment', 'preds_3_label_criteria']).size().reset_index(name='count')

# Calculate the total count for each preds_3_label_criteria group
df_counts_3labels['total_3labels'] = df_counts_3labels.groupby('preds_3_label_criteria')['count'].transform('sum')

# Calculate the proportion for each age_segment within the entire group (Highly, Moderately, Non-Addicted)
df_counts_3labels['proportion'] = df_counts_3labels['count'] / df_counts_3labels['total_3labels']

# Custom palette for hue = 'preds_3_label_criteria'
blue_palette = sns.color_palette("Blues", 8)
custom_palette_3labels = {0: blue_palette[2], 1: blue_palette[5], 2: blue_palette[7]}  # 0 = Non-Addicted, 1 = Mildly Addicted, 2 = Severely Addicted

# Create the bar plot for preds_3_label_criteria using the custom palette
g_3labels = sns.catplot(
    data=df_counts_3labels,  
    x="age_segment", y='proportion', hue='preds_3_label_criteria', 
    order=age_orders, hue_order=[2, 1, 0],
    palette=custom_palette_3labels,  
    height=4.5, aspect=2.15, kind="bar", legend=False
)

# Set labels and formatting
plt.xlabel("", fontsize="20")
plt.ylabel("Proportion", fontsize="23")
plt.xticks(fontsize="25")
plt.yticks(fontsize="22")

g_3labels.ax.spines[['right', 'top']].set_visible(True)

# Get handles and labels from the plot
handles, labels = g_3labels.ax.get_legend_handles_labels()

# Set the custom labels for the legend
plt.legend(handles=handles, labels=['Severely Addicted', 'Mildly Addicted', 'Non-Addicted'], fontsize=22, title_fontsize="18", loc="upper right")

# Adjust layout and save the plot
plt.tight_layout()
plt.savefig('./addiction1_xgboost_20241201revision_output/figs/age_3labels_proportion.pdf', format='pdf', dpi=1000, bbox_inches='tight')
plt.show()

In [ ]:
df_survey_users['age_survey'] = df_survey_users['age_survey'].astype('int64')
df_survey_users['age'] = df_survey_users['age'].astype('int64')


df_survey_users['age_segment'] = df_survey_users['age'].apply(lambda x: separate_age(x, age_orders))
df_survey_users['age_segment_survey'] = df_survey_users['age_survey'].apply(lambda x: separate_age(x, age_orders))


In [ ]:
### Plot 1: preds_soft (Addicted vs Non-Addicted) based on proportions across all age groups
plt.rcParams.update({'font.weight': 'normal','font.size': '20', 'axes.labelsize': 'x-large', 'font.sans-serif': 'Arial'})

# Group by age_segment and preds_soft to calculate the counts for each group
df_counts_soft = df_survey_users.groupby(['age_segment', 'soft_criteria']).size().reset_index(name='count')

# Calculate the total count for each preds_soft group (Addicted and Non-Addicted)
df_counts_soft['total_soft'] = df_counts_soft.groupby('soft_criteria')['count'].transform('sum')

# Calculate the proportion for each age_segment within the entire Addicted/Non-Addicted group
# df_counts_soft['proportion'] = df_counts_soft['count'] / df_counts_soft['total_soft']
df_counts_soft['proportion'] = df_counts_soft['count'] / df_counts_soft['total_soft']

# Custom palette for hue = 'preds_soft' (Addicted vs Non-Addicted)
blue_palette = sns.color_palette("Blues", 6)
custom_palette_soft = {0: blue_palette[1], 1: blue_palette[4]}  # 0 = Non-Addicted, 1 = Addicted

plt.figure(figsize=(20, 3))
# Create the bar plot for preds_soft (Addicted vs Non-Addicted) using the custom palette
g_soft = sns.catplot(
    data=df_counts_soft,  
    x="age_segment", y='proportion', hue='soft_criteria', 
    order=age_orders, hue_order=[1, 0],
    palette=custom_palette_soft,  
    height=4.8, aspect=2.05, kind="bar", legend=False
)

# Set labels and formatting
# plt.xlabel("Age", fontsize="20")
plt.xlabel("", fontsize="22")

plt.ylabel("Proportion", fontsize="21")
plt.xticks(fontsize="25")
plt.yticks(fontsize="22")

# Get handles and labels from the plot
handles, labels = g_soft.ax.get_legend_handles_labels()
g_soft.ax.spines[['right', 'top']].set_visible(True)

# Set the custom labels for Addicted and Non-Addicted
plt.legend(handles=handles, labels=['Addicted', 'Non-Addicted'], fontsize=22, loc="upper right")

# Adjust layout and save the plot
plt.tight_layout()
plt.savefig('./addiction1_xgboost_20241201revision_output/figs/survey_age_soft_proportion.pdf', format='pdf', dpi=1000, bbox_inches='tight')
plt.show()


In [ ]:
df_survey_users.columns

In [ ]:
df_combined_all.groupby('preds_soft')['coverage_per_2weeks'].mean()

In [ ]:
green_palette = sns.color_palette("Greens", 7)
plt.rcParams.update({'font.weight': 'normal','font.size': '18', 'axes.labelsize': 'x-large', 'font.sans-serif': 'Arial'})


# Custom palette for both plots, using same shades of green
custom_palette_green_soft = {0: green_palette[1], 1: green_palette[5]}  # Non-Addicted = 0, Addicted = 1
custom_palette_green_3labels = {0: green_palette[1], 1: green_palette[4], 2: green_palette[6]}  # 0 = Non-Addicted, 1 = Mildly Addicted, 2 = Severely Addicted

### Plot 1: preds_soft (Addicted vs Non-Addicted) based on proportions across genders

# Group by gender and preds_soft to calculate the counts for each group
df_gender_soft = df_combined_month0.groupby(['gender', 'preds_soft']).size().reset_index(name='count')

# Calculate the total count for each preds_soft group (Addicted and Non-Addicted)
df_gender_soft['total_soft'] = df_gender_soft.groupby('preds_soft')['count'].transform('sum')

# Calculate the proportion for each gender within the Addicted/Non-Addicted group
df_gender_soft['proportion'] = df_gender_soft['count'] / df_gender_soft['total_soft']

# Set the figure size for the plot
fig, ax = plt.subplots(figsize=(6.5, 5))

# Create the bar plot for preds_soft (Addicted vs Non-Addicted) using the green palette
sns.barplot(data=df_gender_soft, x="gender", y='proportion', hue='preds_soft', hue_order=[1, 0], alpha=0.8,
            palette=custom_palette_green_soft, capsize=.15, ax=ax)

# Set x-axis labels for gender
plt.xticks(ticks=[0, 1],fontsize='27', labels=['Male', 'Female'])

# Set labels and formatting
plt.xlabel("", fontsize="27")
plt.ylabel("Proportion", fontsize="25")
plt.ylim(0,0.8)
plt.yticks(fontsize="27")
# Get handles and labels from the ax object
handles, labels = ax.get_legend_handles_labels()
plt.legend(handles=handles, labels=['Addicted', 'Non-Addicted'], fontsize=26, loc="upper right")

# Adjust layout and save the plot
plt.tight_layout()
plt.savefig('./addiction1_xgboost_20241201revision_output/figs/gender_soft_proportion.pdf', format='pdf', dpi=1000, bbox_inches='tight')
plt.show()



### Plot 2: GOODpreds_3_label_criteria (Severely Addicted, Mildly Addicted, Non-Addicted) based on proportions across genders

# Group by gender and preds_3_label_criteria to calculate the counts for each group
df_gender_3labels = df_combined_month0.groupby(['gender', 'preds_3_label_criteria']).size().reset_index(name='count')

# Calculate the total count for each preds_3_label_criteria group
df_gender_3labels['total_3labels'] = df_gender_3labels.groupby('preds_3_label_criteria')['count'].transform('sum')

# Calculate the proportion for each gender within the Highly/Moderately/Non-Addicted group
df_gender_3labels['proportion'] = df_gender_3labels['count'] / df_gender_3labels['total_3labels']

# Set the figure size for the plot
fig, ax = plt.subplots(figsize=(6,6))

# Create the bar plot for preds_3_label_criteria using the green palette
sns.barplot(data=df_gender_3labels, x="gender", y='proportion', hue='preds_3_label_criteria', hue_order=[2, 1, 0], alpha=0.8,
            palette=custom_palette_green_3labels, capsize=.15, ax=ax)

# Set x-axis labels for gender
plt.xticks(ticks=[0, 1],fontsize=25, labels=['Male', 'Female'])

# Set labels and formatting
plt.xlabel("", fontsize="20")
plt.ylabel("Proportion", fontsize="23")
plt.yticks(fontsize="22")
plt.ylim(0,1)
handles, labels = ax.get_legend_handles_labels()

# Set custom labels for the legend
plt.legend(handles=handles, labels=['Severely Addicted', 'Mildly Addicted', 'Non-Addicted'], 
        fontsize=22, loc="upper center")

# Adjust layout and save the plot
plt.tight_layout()
plt.savefig('./addiction1_xgboost_20241201revision_output/figs/gender_3labels_proportion.pdf', format='pdf', dpi=1000, bbox_inches='tight')
plt.show()

In [ ]:
# Custom palette for both plots, using same shades of green
custom_palette_green_soft = {0: green_palette[1], 1: green_palette[5]}  # Non-Addicted = 0, Addicted = 1
custom_palette_green_3labels = {0: green_palette[1], 1: green_palette[4], 2: green_palette[6]}  # 0 = Non-Addicted, 1 = Mildly Addicted, 2 = Severely Addicted

### Plot 1: preds_soft (Addicted vs Non-Addicted) based on proportions across genders

# Group by gender and preds_soft to calculate the counts for each group
df_gender_soft = df_survey_users.groupby(['gender', 'soft_criteria']).size().reset_index(name='count')

# Calculate the total count for each preds_soft group (Addicted and Non-Addicted)
df_gender_soft['total_soft'] = df_gender_soft.groupby('soft_criteria')['count'].transform('sum')

# Calculate the proportion for each gender within the Addicted/Non-Addicted group
df_gender_soft['proportion'] = df_gender_soft['count'] / df_gender_soft['total_soft']

# Set the figure size for the plot
fig, ax = plt.subplots(figsize=(6, 5))

# Create the bar plot for preds_soft (Addicted vs Non-Addicted) using the green palette
sns.barplot(data=df_gender_soft, x="gender", y='proportion', hue='soft_criteria', hue_order=[1, 0], alpha=0.8,
            palette=custom_palette_green_soft, capsize=.15, ax=ax)

# Set x-axis labels for gender
plt.xticks(ticks=[0, 1], labels=['Male', 'Female'],fontsize=22)
plt.yticks( fontsize=22)
# Set labels and formatting
plt.xlabel("", fontsize="22")
plt.ylabel("Proportion", fontsize="22")
plt.ylim(0,0.8)
# Get handles and labels from the ax object
handles, labels = ax.get_legend_handles_labels()
plt.legend(handles=handles, labels=['Addicted', 'Non-Addicted'], fontsize=20, loc="upper left")

# Adjust layout and save the plot
plt.tight_layout()
plt.savefig('./addiction1_xgboost_20241201revision_output/figs/survey_gender_soft_proportion.pdf', format='pdf', dpi=1000, bbox_inches='tight')
plt.show()


In [ ]:
# df_gender = df_combined_all.groupby(['gender', 'preds_3_label_criteria']).size().reset_index(name='count')

# # Calculate the total count per gender
# df_gender['total'] = df_gender.groupby('gender')['count'].transform('sum')

# # Calculate the proportion of each preds_3_label_criteria within each gender
# df_gender['proportion'] = df_gender['count'] / df_gender['total']

# # Set the figure size for the plot
# fig, ax = plt.subplots(figsize=(8, 7))

# # Create the bar plot using sns.barplot, showing proportions for each gender and preds_3_label_criteria
# sns.barplot(data=df_gender, x="gender", y='proportion', hue='preds_3_label_criteria', capsize=.15)

# # Set x-axis labels for gender
# plt.xticks(ticks=[0, 1], labels=['Male', 'Female'])

# # Set labels and formatting
# plt.xlabel("Gender", fontweight='bold', fontsize="22")
# plt.ylabel("Proportion", fontweight='bold', fontsize="22")
# plt.ylim(0, 1)

# # Adjust layout and save the plot
# plt.tight_layout()
# plt.savefig('./addiction1_xgboost_20241201revision_output/figs/gender_bar.pdf', format='pdf', dpi=1000, bbox_inches='tight')
# plt.show()

In [ ]:
def get_sig(r_):
    if r_.pvalue < 0.001:   
        return '***'
    elif r_.pvalue < 0.01:
        return '**'
    elif r_.pvalue<0.05:
        return '*'
    else:
        return "Not Significant"

In [ ]:
hour_columns = [col for col in df_combined_month0.columns if "hour" in col]
mean_watch_time = df_combined_month0.groupby("preds_soft")[hour_columns].mean()

# Convert watch time from seconds to minutes
mean_watch_time = mean_watch_time / 60

# Transpose the data for plotting
mean_watch_time = mean_watch_time.T
mean_watch_time.index = [col.replace("hour_", "").replace("_wt", "") for col in hour_columns]  # Simplify x-axis labels

# Define improved custom colors for shading
# custom_palette = {
#   0: "#feb29b",  
#   1: "#74AED4"
# }


# custom_palette = {1: "#a5c1a3",  
#                   0: "#feb29b",  
#                   2: "#74AED4"}  

custom_palette = {
    1: "#a5c1a3",  
  0: "#feb29b",  
    2: "#74AED4"
}
    
# Enhanced Line Plot with Shading
plt.figure(figsize=(11, 3.7))

# Addicted group
plt.plot(
    mean_watch_time.index, mean_watch_time[1], 
    color=custom_palette[2], label="Addicted", linewidth=1.8, alpha=0.8
)
plt.fill_between(
    mean_watch_time.index, 
    mean_watch_time[1], 
    color=custom_palette[2], 
    alpha=0.6  # Slightly opaque shading
)

# Non-Addicted group
plt.plot(
    mean_watch_time.index, mean_watch_time[0], 
    color=custom_palette[0], label="Non-Addicted", linewidth=1.8, alpha=1
)
plt.fill_between(
    mean_watch_time.index, 
    mean_watch_time[0], 
    color=custom_palette[0], 
    alpha=0.9  # Slightly opaque shading
)

# Labels and title
# plt.title("Average Hourly Watch Time", fontsize=16, fontweight='bold', pad=15)
plt.xlabel("Hour of Day", fontsize=20)
plt.ylabel("Watch Time (min)", fontsize=20)

# Ensure x-ticks are regular font
plt.xticks(range(len(hour_columns)), mean_watch_time.index, fontsize=21, style='normal')
plt.yticks(fontsize=21)
plt.xlim(0,23)
plt.ylim(0,12)
legend_handles = [
    Patch(color=custom_palette[2], alpha = 0.7, label="Addicted", lw=1),
    Patch(color=custom_palette[0],alpha=0.7, label="Non-Addicted", lw=1)
         # Thicker line
]
plt.legend(handles=legend_handles, fontsize=22, title="", loc='upper left')

# Tight layout for better spacing
plt.tight_layout()

plt.savefig(f'./addiction1_xgboost_20241201revision_output/figs/hourly_soft.pdf', format='pdf', dpi=900, bbox_inches='tight')
 
# Show plot
plt.show()


hour_columns = [col for col in df_combined_month0.columns if "hour" in col]
mean_watch_time = df_combined_month0.groupby("preds_3_label_criteria")[hour_columns].mean()

# Convert watch time from seconds to minutes
mean_watch_time = mean_watch_time / 60

# Transpose the data for plotting
mean_watch_time = mean_watch_time.T
mean_watch_time.index = [col.replace("hour_", "").replace("_wt", "") for col in hour_columns]  # Simplify x-axis labels

# # Define improved custom colors for shadingGood
# custom_palette = {
#     0: "#c2d9cb",  # Soft Green for Non-Addicted
#     1: "#f2a7ad",  # Soft Purple for Mildly Addicted
#     2: "#fac5a8"   # Sandy Brown for Severely Addicted
# }


# custom_palette = {
#     0: "#c4d9cb",  
#   1: "#feb29b",  
#   2: "#74AED4"
# }


# Enhanced Line Plot with Shading
plt.figure(figsize=(11, 3.7))


# Severely Addicted group
plt.plot(
    mean_watch_time.index, mean_watch_time[2], 
    color=custom_palette[2], label="Severely Addicted", linewidth=1.8, alpha=0.8
)
plt.fill_between(
    mean_watch_time.index, 
    mean_watch_time[2], 
    color=custom_palette[2], 
    alpha=0.6
)



# Mildly Addicted group
plt.plot(
    mean_watch_time.index, mean_watch_time[1], 
    color=custom_palette[1], label="Mildly Addicted", linewidth=1.8, alpha=0.8
)
plt.fill_between(
    mean_watch_time.index, 
    mean_watch_time[1], 
    color=custom_palette[1], 
    alpha=0.9
)



# Non-Addicted group
plt.plot(
    mean_watch_time.index, mean_watch_time[0], 
    color=custom_palette[0], label="Non-Addicted", linewidth=1.8, alpha=1
)
plt.fill_between(
    mean_watch_time.index, 
    mean_watch_time[0], 
    color=custom_palette[0], 
    alpha=0.9
)

# Labels and title
plt.xlabel("Hour of Day", fontsize=21)
plt.ylabel("Watch Time (min)", fontsize=21)

# Ensure x-ticks are regular font
plt.xticks(range(len(hour_columns)), mean_watch_time.index, fontsize=21, style='normal')
plt.yticks(fontsize=21)
plt.xlim(0,23)
plt.ylim(0,11)
# Custom legend with thicker lines
# legend_handles = [

#     Line2D([0], [0], color=custom_palette[2], lw=2.5, label="Severely Addicted"),
#     Line2D([0], [0], color=custom_palette[1], lw=2.5, label="Mildly Addicted"),
#     Line2D([0], [0], color=custom_palette[0], lw=2.5, label="Non-Addicted")

# ]
legend_handles = [
    Patch(color=custom_palette[2], alpha = 0.7, label="Severely Addicted", lw=1),
    Patch(color=custom_palette[1], alpha=0.7,label="Mildly Addicted", lw=1),
    Patch(color=custom_palette[0],alpha=0.9, label="Non-Addicted", lw=1)
]

plt.legend(handles=legend_handles, fontsize=16, title="", title_fontsize=19, loc='upper left')

# Tight layout for better spacing
plt.tight_layout()

# Save the plot
plt.savefig(f'./addiction1_xgboost_20241201revision_output/figs/hourly_preds_3_label_criteria.pdf', format='pdf', dpi=900, bbox_inches='tight')

# Show plot
plt.show()


In [ ]:
hour_columns = [col for col in df_survey_users.columns if "hour" in col]
mean_watch_time = df_survey_users.groupby("soft_criteria")[hour_columns].mean()

# Convert watch time from seconds to minutes
mean_watch_time = mean_watch_time / 60

# Transpose the data for plotting
mean_watch_time = mean_watch_time.T
mean_watch_time.index = [col.replace("hour_", "").replace("_wt", "") for col in hour_columns]  # Simplify x-axis labels

# Define improved custom colors for shading
# custom_palette = {
#   0: "#feb29b",  
#   1: "#74AED4"
# }


# custom_palette = {1: "#a5c1a3",  
#                   0: "#feb29b",  
#                   2: "#74AED4"}  

custom_palette = {
    1: "#a5c1a3",  
  0: "#feb29b",  
    2: "#74AED4"
}



    
# Enhanced Line Plot with Shading
plt.figure(figsize=(10, 4))

# Addicted group
plt.plot(
    mean_watch_time.index, mean_watch_time[1], 
    color=custom_palette[2], label="Addicted", linewidth=1.8, alpha=0.8
)
plt.fill_between(
    mean_watch_time.index, 
    mean_watch_time[1], 
    color=custom_palette[2], 
    alpha=0.6  # Slightly opaque shading
)

# Non-Addicted group
plt.plot(
    mean_watch_time.index, mean_watch_time[0], 
    color=custom_palette[0], label="Non-Addicted", linewidth=1.8, alpha=1
)
plt.fill_between(
    mean_watch_time.index, 
    mean_watch_time[0], 
    color=custom_palette[0], 
    alpha=0.9  # Slightly opaque shading
)

# Labels and title
# plt.title("Average Hourly Watch Time", fontsize=16, fontweight='bold', pad=15)
plt.xlabel("Hour of Day", fontsize=20)
plt.ylabel("Watch Time (Minutes)", fontsize=20)

# Ensure x-ticks are regular font
plt.xticks(range(len(hour_columns)), mean_watch_time.index, fontsize=16, style='normal')
plt.yticks(fontsize=16)
plt.xlim(0,23)
plt.ylim(0,11)
legend_handles = [
    Patch(color=custom_palette[2], alpha = 0.7, label="Addicted", lw=1),
    Patch(color=custom_palette[0],alpha=0.7, label="Non-Addicted", lw=1)
         # Thicker line
]
plt.legend(handles=legend_handles, fontsize=16, title="", title_fontsize=16, loc='upper left')

# Tight layout for better spacing
plt.tight_layout()

plt.savefig(f'./addiction1_xgboost_20241201revision_output/figs/survey_hourly_soft.pdf', format='pdf', dpi=900, bbox_inches='tight')
 
# Show plot
plt.show()


hour_columns = [col for col in df_survey_users.columns if "hour" in col]
mean_watch_time = df_survey_users.groupby("3_label_criteria")[hour_columns].mean()

# Convert watch time from seconds to minutes
mean_watch_time = mean_watch_time / 60

# Transpose the data for plotting
mean_watch_time = mean_watch_time.T
mean_watch_time.index = [col.replace("hour_", "").replace("_wt", "") for col in hour_columns]  # Simplify x-axis labels

# # Define improved custom colors for shadingGood
# custom_palette = {
#     0: "#c2d9cb",  # Soft Green for Non-Addicted
#     1: "#f2a7ad",  # Soft Purple for Mildly Addicted
#     2: "#fac5a8"   # Sandy Brown for Severely Addicted
# }


# custom_palette = {
#     0: "#c4d9cb",  
#   1: "#feb29b",  
#   2: "#74AED4"
# }


# Enhanced Line Plot with Shading
plt.figure(figsize=(10, 4))


# Severely Addicted group
plt.plot(
    mean_watch_time.index, mean_watch_time[2], 
    color=custom_palette[2], label="Severely Addicted", linewidth=1.8, alpha=0.8
)
plt.fill_between(
    mean_watch_time.index, 
    mean_watch_time[2], 
    color=custom_palette[2], 
    alpha=0.6
)



# Mildly Addicted group
plt.plot(
    mean_watch_time.index, mean_watch_time[1], 
    color=custom_palette[1], label="Mildly Addicted", linewidth=1.8, alpha=0.8
)
plt.fill_between(
    mean_watch_time.index, 
    mean_watch_time[1], 
    color=custom_palette[1], 
    alpha=0.9
)



# Non-Addicted group
plt.plot(
    mean_watch_time.index, mean_watch_time[0], 
    color=custom_palette[0], label="Non-Addicted", linewidth=1.8, alpha=1
)
plt.fill_between(
    mean_watch_time.index, 
    mean_watch_time[0], 
    color=custom_palette[0], 
    alpha=0.9
)

# Labels and title
plt.xlabel("Hour of Day", fontsize=20)
plt.ylabel("Watch Time (Minutes)", fontsize=20)

# Ensure x-ticks are regular font
plt.xticks(range(len(hour_columns)), mean_watch_time.index, fontsize=16, style='normal')
plt.yticks(fontsize=16)
plt.xlim(0,23)
plt.ylim(0,11)
# Custom legend with thicker lines
# legend_handles = [

#     Line2D([0], [0], color=custom_palette[2], lw=2.5, label="Severely Addicted"),
#     Line2D([0], [0], color=custom_palette[1], lw=2.5, label="Mildly Addicted"),
#     Line2D([0], [0], color=custom_palette[0], lw=2.5, label="Non-Addicted")

# ]
legend_handles = [
    Patch(color=custom_palette[2], alpha = 0.7, label="Severely Addicted", lw=1),
    Patch(color=custom_palette[1], alpha=0.7,label="Mildly Addicted", lw=1),
    Patch(color=custom_palette[0],alpha=0.9, label="Non-Addicted", lw=1)
]

plt.legend(handles=legend_handles, fontsize=16, title="", title_fontsize=16, loc='upper left')

# Tight layout for better spacing
plt.tight_layout()

# Save the plot
plt.savefig(f'./addiction1_xgboost_20241201revision_output/figs/survey_hourly_preds_3_label_criteria.pdf', format='pdf', dpi=900, bbox_inches='tight')

# Show plot
plt.show()


In [ ]:
df_combined_all['preds_soft_criteria'] = df_combined_all['preds_soft'].astype(bool)

# List of hour columns (from hour_0_wt to hour_23_wt)
hours = [f'hour_{i}_wt' for i in range(24)]

# Group by 'preds_soft' (addicted vs non-addicted) and calculate mean watch time per hour
df_grouped = df_combined_all.groupby('preds_soft')[hours].mean().reset_index()

# ------ 1. Line Plot ------

# Reshape the dataframe for line plot (melt it to long format)
df_grouped_melted = df_grouped.melt(id_vars=['preds_soft'], value_vars=hours, 
                                    var_name='Hour', value_name='Watch Time')
plt.figure(figsize=(12, 6))
sns.lineplot(data=df_grouped_melted, x='Hour', y='Watch Time', hue='preds_soft', marker='o')

plt.title('Average Watch Time by Hour: Addicted vs Non-Addicted Users')
plt.xlabel('Hour of the Day')
plt.ylabel('Average Watch Time (seconds)')
plt.xticks(rotation=45)
plt.legend(title='User Type', labels=['Non-Addicted', 'Addicted'])
plt.tight_layout()
plt.show()

In [ ]:

ddd = df_combined_all[['user_id', 'month_chronological_order',  'preds_soft']].sort_values(by=['user_id', 'month_chronological_order'], ascending=False).reset_index()

ddd['preds_soft_shift'] = ddd['preds_soft'].shift(-1)
ddd['preds_soft_shift'] = ddd['preds_soft_shift'].fillna(True)
ddd['shift'] = ddd['preds_soft'] != ddd['preds_soft_shift']
ddd.drop(columns=['preds_soft_shift'], inplace=True)
ddd.loc[ddd['month_chronological_order']==1, 'shift'] = False

user_stable = ddd.groupby('user_id')['shift'].any()
user_stable = user_stable[user_stable==False].index
ddd_ = ddd[ddd['user_id'].isin(user_stable)]
user_add = ddd_[ddd_['preds_soft']==True]['user_id'].unique()
user_non = ddd_[ddd_['preds_soft']==False]['user_id'].unique()

uuu = ddd[ddd['shift']==True].groupby('user_id')['shift'].count()
uuu = uuu[uuu==1].index # length = 1045
ddd_ = ddd[ddd['user_id'].isin(uuu)]
user_non2add = ddd_[(ddd_['shift']==True)&(ddd_['preds_soft']==True)]['user_id'].unique()
user_add2non = ddd_[(ddd_['shift']==True)&(ddd_['preds_soft']==False)]['user_id'].unique()

len(user_add), len(user_non), len(user_non2add), len(user_add2non)

In [ ]:
for item in feature_xgboost:    
    if item not in  [
        'watch_all_per_day_min',
        'session_all_per_day',
        'coverage_per_2weeks',
        'category_count_unique',
        'pid_SHUA_all_per_day',
        'pid_watch_all_per_day',
        'category_normalized_videos',
        'midnight_wt_min',
 'morning_wt_min',
 'noon_wt_min',
 'afternoon_wt_min',
 'evening_wt_min',
        ]:
        continue
    df_plot = []

    index = np.array(ddd[(ddd['preds_soft']==1)&(ddd['shift']==False)&(ddd['month_chronological_order']!=1)].index)
    df_plot_ = df_combined_all.loc[ddd.loc[index, 'index'], item].values - df_combined_all.loc[ddd.loc[index+1, 'index'], item].values
    df_plot_ = pd.DataFrame(df_plot_, columns=[item])
    df_plot_['type'] = 'A-A'
    df_plot_['hue'] = 'unshift'
    df_plot.append(df_plot_)

    index = np.array(ddd[(ddd['preds_soft']==0)&(ddd['shift']==True)].index)
    df_plot_ = df_combined_all.loc[ddd.loc[index, 'index'], item].values - df_combined_all.loc[ddd.loc[index+1, 'index'], item].values
    df_plot_ = pd.DataFrame(df_plot_, columns=[item])
    df_plot_['type'] = 'A-N'
    df_plot_['hue'] = 'shift'
    df_plot.append(df_plot_)

    index = np.array(ddd[(ddd['preds_soft']==1)&(ddd['shift']==True)].index)
    df_plot_ = df_combined_all.loc[ddd.loc[index, 'index'], item].values - df_combined_all.loc[ddd.loc[index+1, 'index'], item].values
    df_plot_ = pd.DataFrame(df_plot_, columns=[item])
    df_plot_['type'] = 'N-A'
    df_plot_['hue'] = 'shift'
    df_plot.append(df_plot_)

    index = np.array(ddd[(ddd['preds_soft']==0)&(ddd['shift']==False)&(ddd['month_chronological_order']!=1)].index)
    df_plot_ = df_combined_all.loc[ddd.loc[index, 'index'], item].values - df_combined_all.loc[ddd.loc[index+1, 'index'], item].values
    df_plot_ = pd.DataFrame(df_plot_, columns=[item])
    df_plot_['type'] = 'N-N'
    df_plot_['hue'] = 'unshift'
    df_plot.append(df_plot_)

    df_plot = pd.concat(df_plot)

    # gap = [0, 0.2, 0]
    fig, ax = plt.subplots(figsize=(6, 4))
    sns.barplot(data=df_plot, x=item, y='type', orient='h', ax=ax)
    # plt.xlabel(dict_label[item].capitalize())
    plt.xlabel('')
    plt.xticks(fontsize=25)
    plt.yticks(fontsize=25)


    plt.ylabel('State Transition')
    plt.savefig(f'./addiction1_xgboost_20241201revision_output/figs/transition_{item}.pdf', format='pdf',bbox_inches='tight')
    # break

In [ ]:
index = np.array(ddd[(ddd['preds_soft']==1)&(ddd['shift']==True)].index)
df_combined_all.loc[ddd.loc[index, 'index'], ['session_all_per_day', 'user_id', 'month_chronological_order']]

In [ ]:
index = np.array(ddd_[(ddd_['shift']==True)&(ddd_['preds_soft']==True)].index)
df_combined_all.head()

In [ ]:
# for item in ['age','watch_all_per_day_min', 'session_all_per_day', 'midnight_wt_min', 'coverage_per_2weeks', 'category_count_unique']:
import warnings
warnings.filterwarnings("ignore")

for item in feature_xgboost:

    # plt.figure(figsize=(7, 4))
    fig, ax = plt.subplots(figsize=(7, 4))
    custom_palette = {1: "#a5c1a3",  
                  0: "#feb29b",  
                  2: "#74AED4"}  
    
    # Plot density with filled areas for each group
    sns.kdeplot(df_combined_month0[df_combined_month0['preds_soft'] == 1][item], shade=True, color=custom_palette[2], zorder=2,label='Addicted', alpha=0.5, ax=ax)
    sns.kdeplot(df_combined_month0[df_combined_month0['preds_soft'] == 0][item], shade=True, color=custom_palette[0], zorder=1,label='Non-Addicted', alpha=0.7, ax=ax)
    
    # Set labels and title
    if dict_label[item] == 'Daily Exposed Videos':
        plt.xlabel(dict_label[item].capitalize(),fontsize=20)
        plt.xlim(0,2000)
        plt.ylabel('Density',fontsize=20)

    elif dict_label[item] == 'Daily Watched Videos':
        plt.xlabel(dict_label[item].capitalize(),fontsize=20)
        plt.xlim(0,800)
        plt.ylabel('Density',fontsize=20)

    elif dict_label[item] == 'Daily Watch Time (Minutes)':
        plt.xlabel(dict_label[item].capitalize(),fontsize=20)
        plt.xlim(0,300)
        plt.ylabel('Density',fontsize=20)

    else:

        plt.xlabel(dict_label[item],fontsize=20)
        plt.xlim(0,)
        plt.ylabel('Density',fontsize=20)
    # plt.title(f'Density Plot of {dict_label[item]}')
  
    plt.legend(fontsize=16)
    # plt.subplots_adjust(left=0, bottom=0, right=1.2, top=1,)
    # plt.ylabel('Density', labelpad=36)
    plt.subplots_adjust(left=0.1, right=1, top=0.9, bottom=0.1)
    # if item =='coverage_per_2weeks':
    #     plt.legend(fontsize=18, loc='upper left')
    # plt.savefig(f'./addiction1_xgboost_20241201revision_output/figs/density_plot_{item}_soft.pdf', format='pdf', dpi=900, bbox_inches='tight')
    plt.show()

In [ ]:
# df_combined_month0[(df_combined_month0['preds_soft'] == 1)][item]
(df_survey_users['user_id'].unique())
df_combined_month0['user_id'].isin(df_survey_users['user_id'].unique())

In [ ]:
# for item in feature_xgboost:
#     if item not in  [
#         'watch_all_per_day_min',
#         'session_all_per_day',
#         'coverage_per_2weeks',
#         'category_count_unique',
#         'pid_SHUA_all_per_day',
#         'pid_watch_all_per_day',
#         ]:
#         continue
#     # plt.figure(figsize=(7, 4))
#     fig, ax = plt.subplots(figsize=(7, 4))
#     custom_palette = {1: "#a5c1a3",  
#                   0: "#feb29b",  
#                   2: "#74AED4"}  
    
#     # Plot density with filled areas for each group
#     sns.kdeplot(df_combined_month0[df_combined_month0['preds_soft'] == 1][item], shade=True, color=custom_palette[2], zorder=2,label='Addicted', alpha=0.5, ax=ax)
#     sns.kdeplot(df_combined_month0[df_combined_month0['preds_soft'] == 0][item], shade=True, color=custom_palette[0], zorder=1,label='Non-Addicted', alpha=0.7, ax=ax)
    
  
  
#     plt.legend(fontsize=16)
#     # plt.subplots_adjust(left=0, bottom=0, right=1.2, top=1,)
#     # plt.ylabel('Density', labelpad=36)
#     plt.xlabel('')
#     plt.xticks(fontsize=20)
#     plt.subplots_adjust(left=0.1, right=1, top=0.9, bottom=0.1)
#     # if item =='coverage_per_2weeks':
#     #     plt.legend(fontsize=18, loc='upper left')
#     plt.savefig(f'./addiction1_xgboost_20241201revision_output/figs/density_plot_{item}_soft.pdf', format='pdf', dpi=900, bbox_inches='tight')
#     plt.show()

In [ ]:
# for item in ['age','watch_all_per_day_min', 'session_all_per_day', 'midnight_wt_min', 'coverage_per_2weeks', 'category_count_unique']:
import warnings
warnings.filterwarnings("ignore")

for item in feature_xgboost:
    
    # plt.figure(figsize=(7, 4))
    fig, ax = plt.subplots(figsize=(8, 3.7))
    custom_palette = {1: "#a5c1a3",  
                  0: "#feb29b",  
                  2: "#74AED4"}  
    
    # Plot density with filled areas for each group
    sns.kdeplot(df_combined_month0[df_combined_month0['preds_soft'] == 1][item], shade=True, color=custom_palette[2], zorder=2,label='Addicted', alpha=0.5, ax=ax)
    sns.kdeplot(df_combined_month0[df_combined_month0['preds_soft'] == 0][item], shade=True, color=custom_palette[0], zorder=1,label='Non-Addicted', alpha=0.7, ax=ax)
    
    plt.legend(fontsize=27)


    # Set labels and title
    if dict_label[item] == 'Daily Exposed Videos':
        plt.xlabel(dict_label[item].capitalize(),fontsize=20)
        plt.xlim(0,1100,300)
        plt.ylabel('Density',fontsize=20)

    elif dict_label[item] == 'Daily Watched Videos':
        plt.xlabel(dict_label[item].capitalize(),fontsize=20)
        plt.xlim(0,480,200)
        plt.ylabel('Density',fontsize=20)
    elif item == 'evening_wt_min':
        plt.xlabel(dict_label[item].capitalize(),fontsize=20)
        plt.xlim(0,90,30)
        plt.ylabel('Density',fontsize=20)
    elif item == 'midnight_wt_min':
        plt.xlabel(dict_label[item].capitalize(),fontsize=20)
        plt.xlim(0,90,20)
        plt.ylabel('Density',fontsize=20)

    elif item == 'watch_all_per_day_min':
        plt.xlabel(dict_label[item].capitalize(),fontsize=20)
        plt.xlim(0,180,40)
    elif item == 'coverage_per_2weeks':
        plt.xlabel(dict_label[item].capitalize(),fontsize=20)
        plt.xlim(0,13,5)
        plt.legend(fontsize=27,loc='upper left')
    elif item == 'session_all_per_day':
        plt.xlabel(dict_label[item].capitalize(),fontsize=20)
        plt.xlim(0,25,6)
    elif item == 'category_count_unique':
        plt.xlim(0,400,200)
    elif item == 'category_normalized_videos':
        plt.xlabel(dict_label[item].capitalize(),fontsize=20)
        plt.xlim(0,60)
    else:

        plt.xlabel(dict_label[item],fontsize=27)
        plt.xlim(0,)
    # plt.title(f'Density Plot of {dict_label[item]}')
  
    # plt.subplots_adjust(left=0, bottom=0, right=1.2, top=1,)
    # plt.ylabel('Density', labelpad=36)
    plt.xlabel('',fontsize=28)
    plt.ylabel('Density',fontsize=26)

    plt.xticks(fontsize=28)
    plt.yticks(fontsize=26)
  

    # plt.subplots_adjust(left=0.1, right=1, top=0.9, bottom=0.1)
    # if item =='coverage_per_2weeks':
    #     plt.legend(fontsize=18, loc='upper left')
    plt.savefig(f'./addiction1_xgboost_20241201revision_output/figs/density_plot_{item}_soft.pdf', format='pdf', dpi=900, bbox_inches='tight')
    plt.show()

In [ ]:
date_range = pd.date_range(end='2023-09-10', periods=11, freq='30D')
date_range_ = pd.date_range(end='2023-09-11', periods=11, freq='30D')
[(x, y) for x, y in zip(date_range_[:-1], date_range[1:])]

In [ ]:
df_combined_all.columns

In [ ]:

custom_palette = {
    1: "#a5c1a3",  
  0: "#feb29b",  
    2: "#74AED4"
}


# Loop over the feature columns preds_3_label_criteriaGood
# for item in ['age','watch_all_per_day_min', 'session_all_per_day', 'midnight_wt_min', 'coverage_per_2weeks', 'category_count_unique']:
for item in feature_all:
    plt.figure(figsize=(8, 4))
    
    # Plot Non-Addicted first with lower zorder
    sns.kdeplot(
        data=df_combined_month0[df_combined_month0['preds_3_label_criteria'] == 0],
        x=item,
        shade=True,
        color=custom_palette[0],
        label='Non-Addicted',
        alpha=0.8,
        zorder=1
    )
    
    # Plot Mildly Addicted second with medium zorder
    sns.kdeplot(
        data=df_combined_month0[df_combined_month0['preds_3_label_criteria'] == 1],
        x=item,
        shade=True,
        color=custom_palette[1],
        label='Mildly Addicted',
        alpha=0.7,
        zorder=2
    )
    
    # Plot Severely Addicted last with highest zorder
    sns.kdeplot(
        data=df_combined_month0[df_combined_month0['preds_3_label_criteria'] == 2],
        x=item,
        shade=True,
        color=custom_palette[2],
        label='Severely Addicted',
        alpha=0.5,
        zorder=3
    )
    
    # Set labels and title
    if dict_label[item] == 'Daily Exposed Videos':
        plt.xlabel(dict_label[item].capitalize(),fontsize=20)
        plt.xlim(0,1200)
        plt.ylabel('Density',fontsize=20)

    elif dict_label[item] == 'Daily Watched Videos':
        plt.xlabel(dict_label[item].capitalize(),fontsize=20)
        plt.xlim(0,300)
        plt.ylabel('Density',fontsize=20)
        
    elif item == 'midnight_wt_min':
        plt.xlabel(dict_label[item].capitalize(),fontsize=20)
        plt.xlim(0,110)
        plt.ylabel('Density',fontsize=20)
    elif item == 'evening_wt_min':
        plt.xlabel(dict_label[item].capitalize(),fontsize=20)
        plt.xlim(0,110)
        plt.ylabel('Density',fontsize=20)
    elif item == 'session_all_per_day':
        plt.xlabel(dict_label[item].capitalize(),fontsize=20)
        plt.xlim(0,25,6)
    elif item == 'category_count_unique':
        plt.xlim(0,400,200)

    elif item =='coverage_per_2weeks':
        plt.legend(loc='upper left')
        plt.xlim(0,13)
    elif item == 'watch_all_per_day_min':
        plt.xlabel(dict_label[item].capitalize(),fontsize=20)
        plt.xlim(0,190)
        plt.ylabel('Density',fontsize=20)
    elif dict_label[item] == 'Daily Watch Sessions':
        plt.xlabel(dict_label[item].capitalize(),fontsize=20)
        plt.xlim(0,30)
        plt.ylabel('Density',fontsize=20)
    elif item == 'category_normalized_videos':
        plt.xlabel(dict_label[item].capitalize(),fontsize=20)
        plt.xlim(0,70)
        plt.ylabel('Density',fontsize=20)
    elif item == 'category_normalized_videos':
        plt.xlabel(dict_label[item].capitalize(),fontsize=20)
        plt.xlim(0,70)
    else:

        plt.xlabel(dict_label[item],fontsize=20)
        plt.xlim(0,)
        plt.ylabel('Density',fontsize=20)


    # plt.xlabel(dict_label[item], fontsize=20)
    # plt.xlim(0,)
    # plt.ylabel('Density', fontsize=20)
        
    # plt.title(f'Density Plot of {dict_label[item]}', fontsize=22)
    
    # Customize the legend to have 'Severely Addicted' on top
    handles, labels = plt.gca().get_legend_handles_labels()
    # Define the desired order: Severely Addicted, Mildly Addicted, Non-Addicted
    desired_order = ['Severely Addicted', 'Mildly Addicted', 'Non-Addicted']
    ordered_handles = [handles[labels.index(label)] for label in desired_order]
    ordered_labels = [label for label in desired_order]
    plt.legend(ordered_handles, ordered_labels,fontsize=24)
    if item == 'coverage_per_2weeks':
        plt.legend(loc='upper left')
    ax.spines[['right', 'top']].set_visible(False)
    # Add a vertical line at x=0 for reference (optional)
    plt.axvline(x=0, color='black', linewidth=1)
    plt.xticks(fontsize=28)
    plt.yticks(fontsize=26)
    plt.ylabel('Density',fontsize=24)

    plt.xlabel('')
    # Save and show the plot
    plt.tight_layout()
    plt.savefig(f'./addiction1_xgboost_20241201revision_output/figs/density_plot_{item}_3labels.pdf', format='pdf', dpi=900, bbox_inches='tight')
    plt.show()

In [ ]:
# custom_palette = {0: "#1f77b4",  # Non-Addicted: Blue
#                   1: "#ff7f0e",  # Mildly Addicted: Orange
#                   2: "#2ca02c"}  # Severely Addicted: Green
custom_palette = {0: "#a5c1a3",  
                  1: "#feb29b",  
                  2: "#74AED4"}  


# # preds_3_label_criteriaGood
# for feature in feature_column:
    
#     list_ = []  # Initialize the list for t-test results for each feature
    
#     # Create the ECDF plot with custom colors
#     sns.displot(data=df_combined_month0, x=feature, kind='ecdf', height=5, aspect=1.2, linewidth=3,
#                 hue='preds_3_label_criteria', hue_order=[2, 1, 0], palette=custom_palette, legend=False)

#     sns.despine(left=False, bottom=False, top=False, right=False)
#     ax = plt.gca()
    
#     # Set gridlines
#     ax.yaxis.grid(True, linestyle='--', linewidth=0.7, color='lightgray')
#     ax.xaxis.grid(True, linestyle='--', linewidth=0.7, color='lightgray')

#     # Set axis labels
#     plt.xlabel(dict_label[feature], fontsize=22, )
#     plt.ylabel("CDF", fontsize=22,)

#     # Extract the groups based on preds_3_label_criteria
#     vj = df_combined_month0[df_combined_month0['preds_3_label_criteria'] == 0][feature]  # Non-Addicted
#     vi = df_combined_month0[df_combined_month0['preds_3_label_criteria'] == 1][feature]  # Mildly Addicted
#     vh = df_combined_month0[df_combined_month0['preds_3_label_criteria'] == 2][feature]  # Severely Addicted

#     # Perform Levene’s test for equal variances between groups
#     _, p_value = stats.levene(vi, vj)

#     if p_value < 0.05:
#         print("Unequal variances for", feature)
#         r_ = stats.ttest_ind(vi, vj, equal_var=False)
#         sig = get_sig(r_)

#         # Calculate degrees of freedom for unequal variances
#         df = (
#             (vi.var() / len(vi) + vj.var() / len(vj)) ** 2 /
#             (
#                 (1 / (len(vi) - 1)) * (vi.var() / len(vi)) ** 2 +
#                 (1 / (len(vj) - 1)) * (vj.var() / len(vj)) ** 2
#             )
#         )
#     else:
#         print("Equal variances for", feature)
#         r_ = stats.ttest_ind(vi, vj, equal_var=True)
#         sig = get_sig(r_)

#         # Degrees of freedom for equal variances
#         df = len(vi) + len(vj) - 2

#     # Append the results to the list
#     list_.append([r_.statistic, r_.pvalue, df, sig])

#     # Display results as a DataFrame
#     print(f'\n{feature}\n', pd.DataFrame(list_, columns=['t-statistic', 'p-value', 'df', 'Sig. Level']))

#     legend_labels = ['Severely Addicted', 'Mildly Addicted', 'Non-Addicted']
#     handles = [plt.Line2D([0], [0], color=custom_palette[2], lw=3),
#                plt.Line2D([0], [0], color=custom_palette[1], lw=3),
#                plt.Line2D([0], [0], color=custom_palette[0], lw=3)]
#     # plt.legend(handles, legend_labels,  fontsize=14, title_fontsize="15", loc="lower right")
#     plt.savefig(f'./addiction1_xgboost_20241201revision_output/figs/cdf_{feature}_3labels.pdf', format='pdf', dpi=1000, bbox_inches='tight')    
    

#     # Display the plot
#     plt.show()


In [ ]:
# chenrui add, for 2 label t-test
for feature in feature_xgboost:
    list_ = []  # Initialize the list for t-test results for each feature

    vi = df_combined_month0[df_combined_month0['preds_soft'] == 1][feature]
    vj = df_combined_month0[df_combined_month0['preds_soft'] == 0][feature]

    _, p_value = stats.levene(vi, vj)   # Perform Levene’s test for equal variances between groups, p_value < 0.05 means unequal variances(检验同方差假设)

    if p_value < 0.05:
        print("unequal")
        r_ = stats.ttest_ind(vi, vj, equal_var=False)
        sig = get_sig(r_)  # You need to define the get_sig function

        # Calculate df for unequal variances（异方差假设下自由度计算）
        df = (
            (vi.var() / len(vi) + vj.var() / len(vj)) ** 2 /
            (
                (1 / (len(vi) - 1)) * (vi.var() / len(vi)) ** 2 +
                (1 / (len(vj) - 1)) * (vj.var() / len(vj)) ** 2
            )
        )

        list_.append([r_.statistic, r_.pvalue, df, sig])
    else:
        print("equal")
        r_ = stats.ttest_ind(vi, vj, equal_var=True)
        sig = get_sig(r_)  # You need to define the get_sig function

        # Calculate df for equal variances（同方差假设下自由度计算）
        df = len(vi) + len(vj) - 2

        list_.append([r_.statistic, r_.pvalue, df, sig])

    print('\n' + feature + '\n', pd.DataFrame(list_, columns=['t-statistic', 'p-value', 'df', 'Sig. Level']))


In [ ]:
# chenrui add, for p_value 3 labels
import pingouin as pg

def get_sig_pg(pvalue):
    if pvalue < 0.001:   
        return '***'
    elif pvalue < 0.01:
        return '**'
    elif pvalue<0.05:
        return '*'
    else:
        return "Not Significant"
    
for feature in feature_xgboost:
    list_ = []  # Initialize the list for t-test results for each feature

    vi = df_combined_month0[df_combined_month0['preds_3_label_criteria'] == 0][feature].values
    vj = df_combined_month0[df_combined_month0['preds_3_label_criteria'] == 1][feature].values
    vk = df_combined_month0[df_combined_month0['preds_3_label_criteria'] == 2][feature].values

    # Perform Levene’s test for equal variances between groups
    _, p_value = stats.levene(vi, vj, vk)

    if p_value < 0.05:
        print("unequal")
        avona = pg.welch_anova(dv=feature, between='preds_3_label_criteria', data=df_combined_month0)
        list_.append([avona['F'].values[0], avona['p-unc'].values[0], avona['ddof1'].values[0], avona['ddof2'].values[0], get_sig_pg(avona['p-unc'].values[0])])

    else:
        print("equal")
        avona = pg.anova(dv=feature, between='preds_3_label_criteria', data=df_combined_month0)
        # list.append([avona['F'].values[0], avona['p-unc'].values[0], avona['df1'].values[0], avona['df2'].values[0], get_sig(avona['p-unc'].values[0])])
    print('\n' + feature + '\n', pd.DataFrame(list_, columns=['F-statistic', 'p-value', '组间', '组内', 'Sig. Level']))

In [ ]:
plt.figure(figsize=(8, 4))
# mpl.rcParams['font.weight'] = 'bold'
# mpl.rcParams['axes.linewidth'] = 3
plt.rcParams['font.size'] = 22
custom_palette = {0: "#feb29b",  
                  1: "#74AED4",  
                }  

for feature in feature_xgboost:
    
    list_ = []  # Initialize the list for t-test results for each feature
    
    sns.displot(data=df_survey_users, x=feature, kind='ecdf', height=5, aspect=1.2, linewidth=3, palette=custom_palette, hue='soft_criteria', hue_order=[1, 0],
                legend = False
               )
    sns.despine(left=False, bottom=False, top=False, right=False)
    ax = plt.gca()
    ax.yaxis.grid(True, linestyle='--', linewidth=0.7, color='lightgray')
    ax.xaxis.grid(True, linestyle='--', linewidth=0.7, color='lightgray')
    legend_handles = [Line2D([0], [0], color="#74AED4", lw=3, label='Addicted'),
                  Line2D([0], [0], color="#feb29b", lw=3, label='Non-Addicted')]
    plt.legend(handles=legend_handles, fontsize=18)
    if feature == 'pid_SHUA_all_per_day':
      plt.xlabel(dict_label[item].capitalize(),fontsize=20)
      plt.xlim(0,1500)
      plt.ylabel('Density',fontsize=20)
      # plt.legend(loc='bottom right')
    elif feature == 'pid_watch_all_per_day':
      plt.xlim(0,1000)
      # plt.legend(loc='bottom right')
      
    plt.xticks(fontsize=20)
    plt.xlabel('')
    plt.ylabel("CDF", fontsize=22)
    plt.savefig(f'./addiction1_xgboost_20241201revision_output/figs/survey_cdf_{feature}_soft.pdf', format='pdf', dpi=1000, bbox_inches='tight')    
  
    plt.show()


In [ ]:
plt.figure(figsize=(10, 5))
# mpl.rcParams['font.weight'] = 'bold'
# mpl.rcParams['axes.linewidth'] = 3
plt.rcParams['font.size'] = 22
custom_palette = {0: "#feb29b",  
                  1: "#74AED4",  
                }  
for feature in feature_xgboost:

    ###cdf
    list_ = []  # Initialize the list for t-test results for each feature
    
    sns.displot(data=df_combined_month0, x=feature, kind='ecdf', height=5, aspect=1.2, linewidth=3, palette=custom_palette, hue='preds_soft', hue_order=[1, 0],
                legend = False
               )
    sns.despine(left=False, bottom=False, top=False, right=False)
    ax = plt.gca()
    ax.yaxis.grid(True, linestyle='--', linewidth=0.7, color='lightgray')
    ax.xaxis.grid(True, linestyle='--', linewidth=0.7, color='lightgray')
    plt.xlabel(dict_label[feature], fontsize=22)
    plt.ylabel("CDF", fontsize=22)

    vi = df_combined_month0[df_combined_month0['preds_soft'] == 1][feature]
    vj = df_combined_month0[df_combined_month0['preds_soft'] == 0][feature]

    _, p_value = stats.levene(vi, vj)

    if p_value < 0.05:
        print("unequal")
        r_ = stats.ttest_ind(vi, vj, equal_var=False)
        sig = get_sig(r_)  # You need to define the get_sig function

        # Calculate df for unequal variances
        df = (
            (vi.var() / len(vi) + vj.var() / len(vj)) ** 2 /
            (
                (1 / (len(vi) - 1)) * (vi.var() / len(vi)) ** 2 +
                (1 / (len(vj) - 1)) * (vj.var() / len(vj)) ** 2
            )
        )

        list_.append([r_.statistic, r_.pvalue, df, sig])
    else:
        print("equal")
        r_ = stats.ttest_ind(vi, vj, equal_var=True)
        sig = get_sig(r_)  # You need to define the get_sig function

        # Calculate df for equal variances
        df = len(vi) + len(vj) - 2

        list_.append([r_.statistic, r_.pvalue, df, sig])

    print('\n' + feature + '\n', pd.DataFrame(list_, columns=['t-statistic', 'p-value', 'df', 'Sig. Level']))
#     plt.legend(labels=['Non-Addicted','Addicted'], bbox_to_anchor=(1.01,1.1), ncol=2)
    # plt.savefig(f'./addiction1_xgboost_20241201revision_output/figs/cdf_{feature}_soft.pdf', format='pdf', dpi=1000, bbox_inches='tight')    
    plt.show()


In [ ]:
# df_combined_all[['month','date_start']].groupby('month').max()

In [ ]:
# df_combined_all['month_chronological_order'] = (df_combined_all['month'].max() - df_combined_all['month']) + 1

# # Display the first few rows to verify
# df_combined_all[['month', 'month_chronological_order','date_start']].value_counts()

# df_combined_all.shape,df_combined_all[df_combined_all['month_chronological_order'] == df_combined_all['month_chronological_order']].shape

In [ ]:
plt.rcParams['font.family'] = ['Arial']
plt.rcParams['axes.linewidth'] = 1
# linestyles = ['dotted', '--', '-','dashdot']
linestyles = {'non_addicted': '--', 'soft_addicted': '-', 'hard_addicted': '-'}
alphas = {'non_addicted': 0.6, 'soft_addicted': 0.5, 'hard_addicted': 0.9}
plt.rcParams.update({'font.weight': 'normal','font.size':'18','axes.labelsize': 'x-large','font.sans-serif':'Arial'})

for item in [ 'watch_all_per_day_min',
    ]:

    addicted = df_combined_all[df_combined_all['preds_soft'] == 1]
    
    # Non-addicted users
    non_addicted = df_combined_all[df_combined_all['preds_soft'] == 0]
    
    # Plot average watch time per day over months
    plt.figure(figsize=(3, 2))
    sns.lineplot(data=addicted, x='month', y=item,  linestyle='-',color='black',linewidth=2,ci=None)
    sns.lineplot(data=non_addicted, x='month', y=item,  linestyle='--',color='grey', linewidth=2,ci=None)
    # plt.xlabel('Month',fontsize=20)
    # plt.xticks(color='white')
    # plt.yticks(color='white')
    
    plt.ylabel('',fontsize=20)
    # plt.legend()
    # plt.legend(loc='upper left',fontsize=18)
    ax = plt.gca()
    ax.spines['top'].set_visible(False)  # Hide the top spine
    ax.spines['right'].set_visible(False)  # Hide the right spine
    ax.spines['left'].set_linewidth(1)  # Set the left spine width
    ax.spines['bottom'].set_linewidth(1)  # Set the bottom spine width

    # plt.savefig(f'./addiction1_xgboost_20241201revision_output/figs/temporal_{item}_soft.pdf', format='pdf', dpi=900, bbox_inches='tight')
    
    plt.show()

In [ ]:
# Group data and create plot for user counts per month based on soft addiction criteria
df_monthly_counts_soft = df_combined_all.groupby(['month', 'preds_soft']).size().reset_index(name='count')

# Plot settings
plt.figure(figsize=(7, 4))
sns.lineplot(data=df_monthly_counts_soft, x='month', y='count', hue='preds_soft', hue_order=[1, 0],
             linewidth=3, palette={0: "#FC8D62", 1: "#8DA0CB"}, marker="o")

# Customize plot
plt.xlabel('Month', fontsize=20)
plt.ylabel('User Counts', fontsize=20)
plt.grid(True, alpha=0.4)

# Customize legend
legend_handles = [Line2D([0], [0], color="#8DA0CB", lw=3, label='Addicted'),
                  Line2D([0], [0], color="#FC8D62", lw=3, label='Non-Addicted')]
plt.legend(handles=legend_handles, fontsize=18)

# Adjust spine visibility and width to match the style
ax = plt.gca()
ax.spines['top'].set_visible(False)  # Hide the top spine
ax.spines['right'].set_visible(False)  # Hide the right spine
ax.spines['left'].set_linewidth(1)  # Set the left spine width
ax.spines['bottom'].set_linewidth(1)  # Set the bottom spine width

# Layout adjustments
plt.tight_layout()

# Save the plot
# plt.savefig('./addiction1_xgboost_20241201revision_output/figs/temporal_user_counts_soft_V2.pdf', 
            # format='pdf', dpi=1000, bbox_inches='tight')

# Show the plot
plt.show()

In [ ]:
# same as below code cell..

# df_monthly_counts_soft = df_combined_all.groupby(['month_chronological_order', 'preds_soft']).size().reset_index(name='count')

# # Plot settings
# plt.figure(figsize=(7, 4))
# sns.lineplot(data=df_monthly_counts_soft, x='month_chronological_order', y='count', hue='preds_soft', hue_order=[1, 0],
#              linewidth=3, palette={0: "#FC8D62", 1: "#8DA0CB"}, marker="o")

# # Customize plot
# plt.xlabel('Month', fontsize=20)
# plt.ylabel('User Counts', fontsize=20)
# plt.grid(True, alpha=0.4)

# # Customize legend
# legend_handles = [Line2D([0], [0], color="#8DA0CB", lw=3, label='Addicted'),
#                   Line2D([0], [0], color="#FC8D62", lw=3, label='Non-Addicted')]
# plt.legend(handles=legend_handles, fontsize=18)

# # Adjust spine visibility and width to match the style
# ax = plt.gca()
# ax.spines['top'].set_visible(False)  # Hide the top spine
# ax.spines['right'].set_visible(False)  # Hide the right spine
# ax.spines['left'].set_linewidth(1)  # Set the left spine width
# ax.spines['bottom'].set_linewidth(1)  # Set the bottom spine width

# # Layout adjustments
# plt.tight_layout()

# # Save the plot
# # plt.savefig('./addiction1_xgboost_20241201revision_output/figs/temporal_user_counts_soft_V2.pdf', 
#             # format='pdf', dpi=1000, bbox_inches='tight')

# # Show the plot
# plt.show()

In [ ]:
df_combined_all.groupby('user_id')['month_chronological_order'].nunique()

In [ ]:
users_with_nonzero_all_months = (
    df_combined_all[df_combined_all['pid_SHUA_all_per_day'] > 0]
    .groupby('user_id')['month_chronological_order']
    .nunique()
)
# Keep only users who have data for all months
total_months = df_combined_all['month_chronological_order'].nunique()
users_with_nonzero_all_months = users_with_nonzero_all_months[users_with_nonzero_all_months == total_months].index

# Step 2: Filter the dataset to include only these users
df_filtered_all_months = df_combined_all[df_combined_all['user_id'].isin(users_with_nonzero_all_months)]

# Step 3: Group by month and preds_soft to calculate user counts
df_monthly_counts_soft = (
    df_filtered_all_months
    .groupby(['month_chronological_order', 'preds_soft'])
    .size()
    .reset_index(name='count')
)

# Step 4: Plot user counts by month
plt.figure(figsize=(7, 4))
sns.lineplot(
    data=df_monthly_counts_soft,
    x='month_chronological_order',
    y='count',
    hue='preds_soft',
    hue_order=[1, 0],
    linewidth=3,
    palette={0: "#FC8D62", 1: "#8DA0CB"},
    marker="o"
)

# Customize plot labels and grid
plt.xlabel('Month', fontsize=20)
plt.ylabel('User Counts', fontsize=20)
plt.grid(True, alpha=0.4)

# Customize legend
legend_handles = [
    Line2D([0], [0], color="#8DA0CB", lw=3, label='Addicted'),
    Line2D([0], [0], color="#FC8D62", lw=3, label='Non-Addicted')
]
plt.legend(handles=legend_handles, fontsize=18)

# Adjust spine visibility and width to match the style
ax = plt.gca()
ax.spines['top'].set_visible(False)  # Hide the top spine
ax.spines['right'].set_visible(False)  # Hide the right spine
ax.spines['left'].set_linewidth(1)  # Set the left spine width
ax.spines['bottom'].set_linewidth(1)  # Set the bottom spine width

# Layout adjustments
plt.tight_layout()

# Save the plot (optional)
plt.savefig('./addiction1_xgboost_20241201revision_output/figs/temporal_user_counts_soft_shuaIsNot0.pdf', format='pdf', dpi=1000, bbox_inches='tight')


In [ ]:
# Prepare data
df_monthly_counts_soft = df_combined_all.groupby(['month_chronological_order', 'preds_soft']).size().reset_index(name='count')

# Line Plot
plt.figure(figsize=(7, 4))
sns.lineplot(
    data=df_monthly_counts_soft, 
    x='month_chronological_order', 
    y='count', 
    hue='preds_soft', 
    hue_order=[1, 0],  # Ensure "Addicted" (1) and "Non-Addicted" order
    linewidth=3, 
    palette={0: "#FC8D62", 1: "#8DA0CB"}, 
    marker="o"
)
plt.xlabel('Month', fontsize=14)
plt.ylabel('User Counts', fontsize=14)
plt.ylim(2500,7500)

plt.title('User Counts Over Time', fontsize=16)
plt.grid(alpha=0.4)
plt.legend(title='Group', fontsize=12)
plt.tight_layout()
plt.show()

# Stacked Area Plot
df_pivot_area = df_monthly_counts_soft.pivot(
    index='month_chronological_order', 
    columns='preds_soft', 
    values='count'
).fillna(0)

plt.figure(figsize=(7, 4))
plt.stackplot(
    df_pivot_area.index, 
    df_pivot_area[1],  # Addicted
    df_pivot_area[0],  # Non-Addicted
    labels=['Addicted', 'Non-Addicted'], 
    colors=["#8DA0CB", "#FC8D62"], 
    alpha=0.8
)
plt.xlabel('Month', fontsize=14)
plt.ylabel('User Counts', fontsize=14)
plt.title('User Counts Over Time (Stacked)', fontsize=16)
plt.legend(loc='upper left', fontsize=12)
plt.grid(alpha=0.4)
plt.tight_layout()
plt.show()

# Bar Plot
plt.figure(figsize=(9, 5))
sns.barplot(
    data=df_monthly_counts_soft, 
    x='month_chronological_order', 
    y='count', 
    hue='preds_soft', 
    hue_order=[1, 0],  # Ensure "Addicted" (1) and "Non-Addicted" order
    palette={0: "#FC8D62", 1: "#8DA0CB"}
)
plt.xlabel('Month', fontsize=14)
plt.ylabel('User Counts', fontsize=14)
plt.title('User Counts by Month and Group', fontsize=16)
plt.grid(alpha=0.4)
plt.legend(title='Group', fontsize=12)
plt.tight_layout()
plt.show()

# Heatmap
df_heatmap = df_monthly_counts_soft.pivot(
    index='month_chronological_order', 
    columns='preds_soft', 
    values='count'
).fillna(0)


In [ ]:
# df_monthly_counts_3labels

In [ ]:
plt.rcParams['font.family'] = ['Arial']
plt.rcParams['axes.linewidth'] = 1
# linestyles = ['dotted', '--', '-','dashdot']
linestyles = {'non_addicted': '--', 'soft_addicted': '-', 'hard_addicted': '-'}
alphas = {'non_addicted': 0.6, 'soft_addicted': 0.5, 'hard_addicted': 0.9}
plt.rcParams.update({'font.weight': 'normal','font.size':'18','axes.labelsize': 'x-large','font.sans-serif':'Arial'})

# for i in [1]:

    # addicted = df_monthly_counts_soft[df_monthly_counts_soft['preds_soft'] == 1].groupby('month')[item].count()
    
    # # Non-addicted users
    # non_addicted = df_combined_all[df_combined_all['preds_soft'] == 0].groupby('month')[item].count()
    
addicted = df_monthly_counts_soft[df_monthly_counts_soft['preds_soft'] == 1]
non_addicted = df_monthly_counts_soft[df_monthly_counts_soft['preds_soft'] == 0]

# addicted = df_combined_all[
# (df_combined_all['coverage_per_2weeks'] > 0) 
# & (df_combined_all['preds_soft'] == 1)
# ].groupby('month_chronological_order')['user_id'].count().reset_index().rename(columns={'user_id': 'count'})
# non_addicted = df_combined_all[
# (df_combined_all['coverage_per_2weeks'] > 0)
# & (df_combined_all['preds_soft'] == 0)
# ].groupby('month_chronological_order')['user_id'].count().reset_index().rename(columns={'user_id': 'count'})

# # Plot average watch time per day over months
fig, ax = plt.subplots(figsize=(9, 5))
# sns.lineplot(data=addicted, x='month', y=item, label='Addicted',  linestyle='-',color='#2A5580',linewidth=4,alpha=0.50,ci=None)
# sns.lineplot(data=non_addicted, x='month', y=item, label='Non-Addicted', linestyle='--',color='#2A5580', linewidth=4, alpha=0.6,ci=None)
ax.plot(addicted.month_chronological_order.values, addicted['count'].values, linewidth=4, linestyle='-', color='#2A5580', label='Addicted', alpha=0.99)
ax.plot(non_addicted.month_chronological_order.values, non_addicted['count'].values, linewidth=4, linestyle='--', color='#2A5580', label='Non-Addicted', alpha=0.7)


plt.xlabel('Month', fontsize=32)
plt.ylabel('User Counts', fontsize=30)
plt.xticks(fontsize=35)
plt.yticks(fontsize=32)
# plt.legend()
plt.legend(fontsize=23)
ax = plt.gca()
ax.spines['top'].set_visible(False)  # Hide the top spine
ax.spines['right'].set_visible(False)  # Hide the right spine
ax.spines['left'].set_linewidth(1)  # Set the left spine width
ax.spines['bottom'].set_linewidth(1)  # Set the bottom spine width

plt.savefig('./addiction1_xgboost_20241201revision_output/figs/temporal_user_counts_soft_V2.pdf', 
        format='pdf', dpi=1000, bbox_inches='tight')
    

df_monthly_counts_3labels = df_combined_all.groupby(['month_chronological_order', 'preds_3_label_criteria']).size().reset_index(name='count')

severely = df_monthly_counts_3labels[df_monthly_counts_3labels['preds_3_label_criteria'] == 2]
mildly = df_monthly_counts_3labels[df_monthly_counts_3labels['preds_3_label_criteria'] == 1]
non_addicted = df_monthly_counts_3labels[df_monthly_counts_3labels['preds_3_label_criteria'] == 0]

# severely = df_combined_all[
# (df_combined_all['coverage_per_2weeks'] > 0) 
# & (df_combined_all['preds_3_label_criteria'] == 2)
# ].groupby('month_chronological_order')['user_id'].count().reset_index().rename(columns={'user_id': 'count'})
# mildly = df_combined_all[
# (df_combined_all['coverage_per_2weeks'] > 0)
# & (df_combined_all['preds_3_label_criteria'] == 1)
# ].groupby('month_chronological_order')['user_id'].count().reset_index().rename(columns={'user_id': 'count'})
# non_addicted = df_combined_all[
# (df_combined_all['coverage_per_2weeks'] > 0)
# & (df_combined_all['preds_3_label_criteria'] == 0)
# ].groupby('month_chronological_order')['user_id'].count().reset_index().rename(columns={'user_id': 'count'})

# # Plot average watch time per day over months
fig, ax = plt.subplots(figsize=(8, 5))
# sns.lineplot(data=addicted, x='month', y=item, label='Addicted',  linestyle='-',color='#2A5580',linewidth=4,alpha=0.50,ci=None)
# sns.lineplot(data=non_addicted, x='month', y=item, label='Non-Addicted', linestyle='--',color='#2A5580', linewidth=4, alpha=0.6,ci=None)
ax.plot(severely.month_chronological_order.values, severely['count'].values, linewidth=4, linestyle='-', color='#2A5580', label='Severely Addicted', alpha=0.99)
ax.plot(mildly.month_chronological_order.values, mildly['count'].values, linewidth=4, linestyle='-', color='#2A5580', label='Mildly Addicted', alpha=0.6)
ax.plot(non_addicted.month_chronological_order.values, non_addicted['count'].values, linewidth=4, linestyle='--', color='#2A5580', label='Non-Addicted', alpha=0.5)


plt.xlabel('Month', fontsize=23)
plt.ylabel('User Counts', fontsize=22)
plt.xticks(fontsize=23)
plt.yticks(fontsize=22)

# plt.ylim(0, 10000)
# plt.legend()
plt.legend(fontsize=18, loc='lower right')
ax = plt.gca()
ax.spines['top'].set_visible(False)  # Hide the top spine
ax.spines['right'].set_visible(False)  # Hide the right spine
ax.spines['left'].set_linewidth(1)  # Set the left spine width
ax.spines['bottom'].set_linewidth(1)  # Set the bottom spine width

plt.savefig('./addiction1_xgboost_20241201revision_output/figs/temporal_user_counts_3_labels_V2.pdf', 
        format='pdf', dpi=1000, bbox_inches='tight')

In [ ]:
df_survey_users[df_survey_users['age_survey']<18]['user_id'].unique(),df_combined_all[df_combined_all['age_survey']<18]['user_id'].unique()

In [ ]:
df_combined_all.groupby(['month_chronological_order','preds_soft'])['user_id'].count(),df_combined_all.groupby(['month','preds_soft'])['user_id'].count()

In [ ]:
# 11 - addicted.month.values

In [ ]:
# df_combined_all['date_start'].unique()

In [ ]:
# df_combined_all[['month','date_start','date_end']]

In [ ]:
df_combined_all

In [ ]:
feature_xgboost

In [ ]:
df_combined_month0['coverage_per_2weeks'].describe(),df_combined_all[df_combined_all['month_chronological_order']==10]['coverage_per_2weeks'].describe()

In [ ]:
plt.rcParams['font.family'] = ['Arial']
plt.rcParams['axes.linewidth'] = 1
# linestyles = ['dotted', '--', '-','dashdot']
linestyles = {'non_addicted': '--', 'soft_addicted': '-', 'hard_addicted': '-'}
alphas = {'non_addicted': 0.6, 'soft_addicted': 0.5, 'hard_addicted': 0.9}
plt.rcParams.update({'font.weight': 'normal','font.size':'18','axes.labelsize': 'x-large','font.sans-serif':'Arial'})



date_start_table = []

for item in feature_xgboost:

    # addicted = df_combined_all[
    #     (df_combined_all['coverage_per_2weeks'] > 0) 
    #     & (df_combined_all['preds_soft'] == 1)
    #     ].groupby('month_chronological_order')[item].mean()
    # non_addicted = df_combined_all[
    #     (df_combined_all['coverage_per_2weeks'] > 0)
    #     & (df_combined_all['preds_soft'] == 0)
    #     ].groupby('month_chronological_order')[item].mean()
    addicted = df_combined_all[df_combined_all['preds_soft'] == 1].groupby('month_chronological_order')[item].mean()
    non_addicted = df_combined_all[df_combined_all['preds_soft'] == 0].groupby('month_chronological_order')[item].mean()

    
    # Map month index to corresponding date_start
    # df_combined_all['date_start'] = pd.to_datetime(df_combined_all['date_start'])
    # month_start_date = df_combined_all.groupby('month')['date_start'].min().to_dict()

    # Reverse the months for plotting, but map them back to the original month for date_start
    # month_chronological_orders = addicted.index[::-1] + 1  # This is the reversed month order for the plot

    # Plot average watch time per day over months
    fig, ax = plt.subplots(figsize=(9, 5))
    sns.lineplot(data=addicted, x=addicted.index, y=addicted.values, label='Addicted',  linestyle='-',color='#2A5580',linewidth=4,alpha=0.99,ci=None)
    sns.lineplot(data=non_addicted, x=non_addicted.index, y=non_addicted.values, label='Non-Addicted', linestyle='--',color='#2A5580', linewidth=4, alpha=0.7,ci=None)
    # ax.plot(addicted.index[::-1]+1, addicted.values, linewidth=4, linestyle='-', color='#2A5580', label='All Addicted', alpha=0.99)
    # ax.plot(non_addicted.index[::-1]+1, non_addicted, linewidth=4, linestyle='--', color='#2A5580', label='Non-Addicted', alpha=0.99)

#   # Add the date_start labels on the plot for each month (matching reversed order)
#     for i, month in enumerate(month_chronological_orders):
#         month_date_start = month_start_date.get(month)
#         if month_date_start:
#             # Ensure the label's x-position matches the month in reversed order
#             ax.text(i, addicted.values[i] + 0.02, month_date_start.strftime('%Y-%m-%d'), ha='center', va='bottom', fontsize=10, rotation=45)


    plt.xlabel('Month',fontsize=32)
    plt.xticks(fontsize=35)
    plt.yticks(fontsize=32)

    plt.ylabel(f'{dict_label[item]}',fontsize=30)
    # plt.legend()
    plt.legend(fontsize=23)
    ax = plt.gca()
    ax.spines['top'].set_visible(False)  # Hide the top spine
    ax.spines['right'].set_visible(False)  # Hide the right spine
    ax.spines['left'].set_linewidth(1)  # Set the left spine width
    ax.spines['bottom'].set_linewidth(1)  # Set the bottom spine width

    plt.savefig(f'./addiction1_xgboost_20241201revision_output/figs/temporal_{item}_soft.pdf', format='pdf', dpi=900, bbox_inches='tight')
    
    # plt.show()

    # # Collect month, date_start, and the corresponding addicted and non-addicted values
    # for month_chronological_order in month_chronological_orders:
    #     # Get the corresponding date_start
    #     date_start = month_start_date.get(month_chronological_order)
    #     if date_start:
    #         # Append the data to the table
    #         date_start_table.append([date_start.strftime('%Y-%m-%d'), month_chronological_order, addicted[month_chronological_order], non_addicted[month_chronological_order]])

# # Create a DataFrame from the collected data
# date_start_df = pd.DataFrame(date_start_table, columns=['Date Start', 'Reversed Month', 'Addicted Value', 'Non-Addicted Value'])

# # Display the resulting table
# date_start_df

In [ ]:
df_combined_all['category_normalized_videos'].unique()

In [ ]:
plt.rcParams['font.family'] = ['Arial']
plt.rcParams['axes.linewidth'] = 1
# linestyles = ['dotted', '--', '-','dashdot']
linestyles = {'non_addicted': '--', 'soft_addicted': '-', 'hard_addicted': '-'}
alphas = {'non_addicted': 0.6, 'soft_addicted': 0.5, 'hard_addicted': 0.9}
plt.rcParams.update({'font.weight': 'normal','font.size':'18','axes.labelsize': 'x-large','font.sans-serif':'Arial'})

date_start_table = []

for item in feature_xgboost:

    # addicted = df_combined_all[
    #     (df_combined_all['coverage_per_2weeks'] > 0) 
    #     & (df_combined_all['preds_soft'] == 1)
    #     ].groupby('month_chronological_order')[item].mean()
    # non_addicted = df_combined_all[
    #     (df_combined_all['coverage_per_2weeks'] > 0)
    #     & (df_combined_all['preds_soft'] == 0)
    #     ].groupby('month_chronological_order')[item].mean()
    s_addicted = df_combined_all[df_combined_all['preds_3_label_criteria'] == 2].groupby('month_chronological_order')[item].mean()
    m_addicted = df_combined_all[df_combined_all['preds_3_label_criteria'] == 1].groupby('month_chronological_order')[item].mean()
    non_addicted = df_combined_all[df_combined_all['preds_3_label_criteria'] == 0].groupby('month_chronological_order')[item].mean()

    
    # Map month index to corresponding date_start
    # df_combined_all['date_start'] = pd.to_datetime(df_combined_all['date_start'])
    # month_start_date = df_combined_all.groupby('month')['date_start'].min().to_dict()

    # Reverse the months for plotting, but map them back to the original month for date_start
    # month_chronological_orders = addicted.index[::-1] + 1  # This is the reversed month order for the plot

    # Plot average watch time per day over months
    fig, ax = plt.subplots(figsize=(8, 5))
    sns.lineplot(data=s_addicted, x=s_addicted.index, y=s_addicted.values, label='Severely Addicted',  linestyle='-',color='#2A5580',linewidth=4,alpha=0.99,ci=None)
    sns.lineplot(data=m_addicted, x=m_addicted.index, y=m_addicted.values, label='Mildly Addicted',  linestyle='-',color='#2A5580',linewidth=4,alpha=0.6,ci=None)
    sns.lineplot(data=non_addicted, x=non_addicted.index, y=non_addicted.values, label='Non-Addicted', linestyle='--',color='#2A5580', linewidth=4, alpha=0.99,ci=None)
    # ax.plot(addicted.index[::-1]+1, addicted.values, linewidth=4, linestyle='-', color='#2A5580', label='All Addicted', alpha=0.99)
    # ax.plot(non_addicted.index[::-1]+1, non_addicted, linewidth=4, linestyle='--', color='#2A5580', label='Non-Addicted', alpha=0.99)

#   # Add the date_start labels on the plot for each month (matching reversed order)
#     for i, month in enumerate(month_chronological_orders):
#         month_date_start = month_start_date.get(month)
#         if month_date_start:
#             # Ensure the label's x-position matches the month in reversed order
#             ax.text(i, addicted.values[i] + 0.02, month_date_start.strftime('%Y-%m-%d'), ha='center', va='bottom', fontsize=10, rotation=45)


    plt.xlabel('Month',fontsize=20)
    plt.ylabel(f'{dict_label[item]}',fontsize=20)
    # plt.legend()
    plt.legend(fontsize=18)
    ax = plt.gca()
    ax.spines['top'].set_visible(False)  # Hide the top spine
    ax.spines['right'].set_visible(False)  # Hide the right spine
    ax.spines['left'].set_linewidth(1)  # Set the left spine width
    ax.spines['bottom'].set_linewidth(1)  # Set the bottom spine width

    plt.savefig(f'./addiction1_xgboost_20241201revision_output/figs/temporal_{item}_soft.pdf', format='pdf', dpi=900, bbox_inches='tight')
    
    plt.show()

    # # Collect month, date_start, and the corresponding addicted and non-addicted values
    # for month_chronological_order in month_chronological_orders:
    #     # Get the corresponding date_start
    #     date_start = month_start_date.get(month_chronological_order)
    #     if date_start:
    #         # Append the data to the table
    #         date_start_table.append([date_start.strftime('%Y-%m-%d'), month_chronological_order, addicted[month_chronological_order], non_addicted[month_chronological_order]])

# Create a DataFrame from the collected data
date_start_df = pd.DataFrame(date_start_table, columns=['Date Start', 'Reversed Month', 'Addicted Value', 'Non-Addicted Value'])

# Display the resulting table
date_start_df

In [ ]:
df_combined_all.columns

In [ ]:
items = ['watch_all_per_day_min', 'session_all_per_day', 'midnight_wt_min', 'coverage_per_2weeks', 'category_count_unique']

# Assuming 'gender' and 'age_segment' columns exist in df_combined_all
gender_groups = df_combined_all.groupby('gender')
age_segment_groups = df_combined_all.groupby('age_segment')

# Loop through each item and plot for different gender and age segments
for item in items:
    # Plot for gender
    fig, ax = plt.subplots(figsize=(8, 5))
    for gender, group in gender_groups:
        # Get addicted and non-addicted means by month_chronological_order
        addicted = group[group['preds_soft'] == 1].groupby('month_chronological_order')[item].mean()
        non_addicted = group[group['preds_soft'] == 0].groupby('month_chronological_order')[item].mean()
        if gender == 0:  # Male
            gender_label = 'Male'
            ax.plot(addicted.index, addicted.values, label=f'{gender_label} - Addicted', linestyle='-', linewidth=4)
            ax.plot(non_addicted.index, non_addicted.values, label=f'{gender_label} - Non-Addicted', linestyle='--', linewidth=4)
        else:  # Female
            gender_label = 'Female'
            ax.plot(addicted.index, addicted.values, label=f'{gender_label} - Addicted', linestyle='-', linewidth=4)
            ax.plot(non_addicted.index, non_addicted.values, label=f'{gender_label} - Non-Addicted', linestyle='--', linewidth=4)
        
        # # Plot using month_chronological_order
        # ax.plot(addicted.index, addicted.values, label=f'{gender} - Addicted', linestyle='-', linewidth=4)
        # ax.plot(non_addicted.index, non_addicted.values, label=f'{gender} - Non-Addicted', linestyle='--', linewidth=4)
    

    
    # ax.set_title(f'{item} by Gender (Reversed Months)')
    ax.set_xlabel('Month')
    plt.ylabel(f'{dict_label[item]}',fontsize=20)
    ax.legend(loc='upper center', bbox_to_anchor=(0.5, -0.1), ncol=3, fontsize=12)
    
    plt.show()

    # Plot for age segment
    fig, ax = plt.subplots(figsize=(12, 8))
    for age_segment, group in age_segment_groups:
        # Get addicted and non-addicted means by month_chronological_order
        addicted = group[group['preds_soft'] == 1].groupby('month_chronological_order')[item].mean()
        non_addicted = group[group['preds_soft'] == 0].groupby('month_chronological_order')[item].mean()
        
        # Plot using month_chronological_order
        ax.plot(addicted.index, addicted.values, label=f'{age_segment} - Addicted', linestyle='-', linewidth=4)
        ax.plot(non_addicted.index, non_addicted.values, label=f'{age_segment} - Non-Addicted', linestyle='--', linewidth=4)
    
    # ax.set_title(f'{item} by Age Segment (Reversed Months)')
    ax.set_xlabel('Month')
    plt.ylabel(f'{dict_label[item]}',fontsize=20)

    ax.legend(loc='upper center', bbox_to_anchor=(0.5, -0.1), ncol=3, fontsize=12)
    
    plt.show()

In [ ]:
# df_combined_all.groupby(['preds_3_label_criteria','month'])['watch_all_per_day_min'].mean()

In [ ]:
## preds_3_label_criteriaGood
plt.rcParams['font.family'] = ['Arial']
plt.rcParams['axes.linewidth'] = 1
linestyles = {'non_addicted': '--', 'soft_addicted': '-', 'hard_addicted': '-'}
alphas = {'non_addicted': 0.6, 'soft_addicted': 0.5, 'hard_addicted': 0.9}

plt.rcParams.update({'font.weight': 'normal','font.size':'18','axes.labelsize': 'x-large','font.sans-serif':'Arial'})

for item in [ 'watch_all_per_day_min',
    'session_all_per_day',
    'midnight_wt_min',
    'coverage_per_2weeks',
    'category_count_unique']:
    hard_addicted = df_combined_all[df_combined_all['preds_3_label_criteria'] == 2].groupby('month')[item].mean()

    soft_addicted = df_combined_all[df_combined_all['preds_3_label_criteria'] == 1].groupby('month')[item].mean()
    # Non-addicted users
    non_addicted = df_combined_all[df_combined_all['preds_3_label_criteria'] == 0].groupby('month')[item].mean()
    
    # Plot average watch time per day over months
    fig, ax = plt.subplots(figsize=(7, 5))
    # sns.lineplot(data=hard_addicted, x='month', y=item, label='Severely Addicted',  linestyle= linestyles['hard_addicted'],color='#2A5580',linewidth=4,alpha=alphas['hard_addicted'],ci=None)
    # sns.lineplot(data=soft_addicted, x='month', y=item, label='Mildly Addicted',  linestyle=linestyles['soft_addicted'],color='#2A5580',linewidth=4,alpha=alphas['soft_addicted'],ci=None)
    # sns.lineplot(data=non_addicted, x='month', y=item, label='Non-Addicted', linestyle=linestyles['non_addicted'],color='#2A5580', linewidth=4, alpha=alphas['non_addicted'],ci=None)
    ax.plot(hard_addicted.index[::-1] + 1, hard_addicted.values, linewidth=4, linestyle='-', color='#2A5580', label='Severely Addicted', alpha=0.99)
    ax.plot(soft_addicted.index[::-1] + 1, soft_addicted, linewidth=4, linestyle='-', color='#2A5580', label='Mild Addicted', alpha=0.5)
    ax.plot(non_addicted.index[::-1] + 1, non_addicted, linewidth=4, linestyle='--', color='#2A5580', label='Non-Addicted', alpha=0.5)

    plt.xlabel('Month',fontsize=20)
    plt.ylabel(f'{dict_label[item]}',fontsize=20)
    # plt.legend(loc='upper left', fontsize=14)
    ax = plt.gca()
    ax.spines['top'].set_visible(False)  # Hide the top spine
    ax.spines['right'].set_visible(False)  # Hide the right spine
    ax.spines['left'].set_linewidth(1)  # Set the left spine width
    ax.spines['bottom'].set_linewidth(1)  # Set the bottom spine width
    plt.legend(fontsize=16)
    plt.savefig(f'./addiction1_xgboost_20241201revision_output/figs/temporal_{item}_3labels.pdf', format='pdf', dpi=900, bbox_inches='tight')
    
    plt.show()

In [ ]:
df_stranger_demograph = pd.read_csv('./10000_user_demographics.csv')

In [ ]:
# Fill missing fre_city_level from df_survey_month0 with fre_city_level from df_stranger_demograph
df_combined_month0['fre_city_level'] = df_combined_month0['fre_city_level'].fillna(df_combined_month0['fre_city_level_stranger'])

# Drop the extra fre_city_level_stranger column as it is no longer needed
df_combined_month0 = df_combined_month0.drop(columns=['fre_city_level_stranger'])

In [ ]:
df_combined_month0.columns

In [ ]:
plt.figure(figsize=(14, 6))
fre_city_level_order = ['新一线城市','一线城市','二线城市','三线城市','四线城市','五线城市','unknown']
df_combined_month0['fre_city_level'] = pd.Categorical(df_combined_month0['fre_city_level'], categories=fre_city_level_order, ordered=True)
sns.histplot(df_combined_month0, x='fre_city_level',hue="preds_soft",hue_order=[1,0],multiple='dodge',shrink=0.4,legend=False)

In [ ]:
y = 'preds_soft'
feature_column = [
    "age",
    'gender',
    'watch_all_per_day', 
    'coverage_per_2weeks',
    "category_count_unique",
    'session_all_per_day',
    'midnight_wt',
]
# Fit a logistic regression model
for feature in feature_column:
    logit_mod = smf.logit(formula=f'{y} ~ {feature}', data=df_survey_month0)
    logit_res = logit_mod.fit()

    # Print the results summary
    print("Model =", f'{y} ~ {feature}', '\n', logit_res.summary())

In [ ]:
df_combined_all['month']

In [ ]:
df_root_1w = pd.read_csv('user_root.csv')
# df_root_survey = pd.read_csv('survey_user_root.csv')
df_root_survey = pd.read_csv('ks_output.csv', usecols=['user_id', 'category_id', 'p_date', 'pid','watch_time'])

# suppleM = pd.read_csv('supplementary_materials_data.csv')


In [ ]:
df_root_survey['root_id'] = df_root_survey['category_id']

In [ ]:
df_root_survey = df_root_survey[df_root_survey['watch_time'] > 0]

# Group by user_id, root_id, and p_date, and count the number of interactions (pid)
df_root_survey = df_root_survey.groupby(['user_id', 'root_id', 'p_date']).agg(
    num_interactions=('pid', 'count')
).reset_index()


In [ ]:
df_root_survey.head(2)

In [ ]:
week_start = pd.to_datetime('2023-09-11')
df_root_1w['p_date'] = pd.to_datetime(df_root_1w['p_date'], format='%Y%m%d')
df_root_1w['month'] = ((week_start - df_root_1w['p_date'] ).dt.days // 30).astype(int)
df_root_1w = df_root_1w[df_root_1w['month']<10]
df_root_1w = df_root_1w[df_root_1w['month']>=0]
df_root_1w['month'].value_counts()

df_root_survey['p_date'] = pd.to_datetime(df_root_survey['p_date'], format='%Y%m%d')
df_root_survey['month'] = ((week_start - df_root_survey['p_date'] ).dt.days // 30).astype(int)
df_root_survey = df_root_survey[df_root_survey['month']<10]
df_root_survey = df_root_survey[df_root_survey['month']>=0]
df_root_survey['month'].value_counts()

df_root_1w = df_root_1w.merge(df_combined_all[['preds_soft','preds_hard','preds_3_label_criteria','user_id','month']], on=['user_id','month'], how='left')
df_root_1w.head(2)
df_root_survey = df_root_survey.merge(df_combined_all[['preds_soft','preds_hard','preds_3_label_criteria','user_id','month']], on=['user_id','month'], how='left')
df_root_survey.head(2)

In [ ]:
df_root_1w.head(2)

In [ ]:
df_root_survey['month'].unique()

In [ ]:
df_root_1w.groupby(['user_id','month'])['root_id'].unique()

In [ ]:


start = '2022-11-15'
end = '2023-9-10'

xxx = df_root_1w[(df_root_1w['p_date'] >= start) & (df_root_1w['p_date'] <= end)]

date_range = pd.date_range(start=start, end=end, freq='D')


daily_counts = xxx.groupby('p_date')['user_id'].nunique()


daily_counts = daily_counts.reindex(date_range, fill_value=0)


In [ ]:
daily_counts[daily_counts==0]

In [ ]:
df_combb_root=pd.concat([df_root_1w,df_root_survey])
df_combb_root.shape


In [ ]:
df_combb_root['root_id'].nunique(),df_root_survey['root_id'].nunique(),df_root_1w['root_id'].nunique()

In [ ]:
df_combb_root['root_id'] = df_combb_root['root_id'].replace('nan', np.nan)
df_combb_root['root_id'] = df_combb_root['root_id'].replace('\\N', np.nan)
df_combb_root = df_combb_root.dropna(subset=['root_id'])

# df_combb_root['root_id'] = df_combb_root['root_id'].astype(int)

In [ ]:
df_combb_root['root_id'].nunique(),df_root_survey['root_id'].nunique(),df_root_1w['root_id'].nunique()


In [ ]:
df_root_survey['root_id'].unique(),df_root_1w['root_id'].unique()

In [ ]:
df_combb_root


In [ ]:
df_video_category =  pd.read_csv('./data/video_category_IDtoName_bank.csv',encoding='utf-8')

In [ ]:
df_video_category.head(2)

In [ ]:
df_video_category['root_id'] = df_video_category['root_id'].astype('int64')
df_combb_root['root_id'] = df_combb_root['root_id'].astype('int64')


In [ ]:
df_combb_root =df_combb_root.merge(df_video_category[['root_id','root_name']],on='root_id',how='left')

In [ ]:
df_combb_root[['user_id','root_name', 'num_interactions']].head(), df_combb_root.month.unique(), df_combb_root.shape, df_combb_root['root_name'].nunique()

In [ ]:
df_combb_root['p_date'].describe()

In [ ]:
df_combb_root['month'].unique()

In [ ]:
# df_combb_root['p_date'] = pd.to_datetime(df_combb_root['p_date'], format='%Y%m%d')
df_combb_root = df_combb_root[(df_combb_root['p_date'] >= '2023-06-25') & (df_combb_root['p_date'] <= '2023-10-25')]
df_combb_root.groupby('p_date')['user_id'].count().plot()

In [ ]:
df_combb_root['p_date'].describe()

In [ ]:
df_combb_root.head(2)

In [ ]:
# Set styling for the plot
plt.rcParams['font.sans-serif'] = ['SimHei', 'Arial Unicode MS', 'STFangsong', 'sans-serif']
plt.rcParams['axes.unicode_minus'] = False  # Handle minus signs properly
mpl.rcParams['axes.linewidth'] = 1
plt.rcParams['font.size'] = 20


custom_palette = {
    0: "#a5c1a3",  
  1: "#feb29b",  
                  2: "#74AED4"
}


# Calculate sum and count of interactions for addicted and non-addicted groups
df_result_0 = df_combb_root[df_combb_root['preds_soft'] == 0].groupby(['root_name'])['num_interactions'].agg(['sum', 'count'])
df_result_1 = df_combb_root[df_combb_root['preds_soft'] == 1].groupby(['root_name'])['num_interactions'].agg(['sum', 'count'])

# Calculate the average interactions for each root_name
df_result_0['divided'] = df_result_0['sum'] / df_result_0['count']
df_result_1['divided'] = df_result_1['sum'] / df_result_1['count']

# Sort by average interactions and get the top 5 categories
top_root_ids_0 = df_result_0['divided'].sort_values(ascending=False).iloc[:5].index
top_root_ids_1 = df_result_1['divided'].sort_values(ascending=False).iloc[:5].index

# Combine top categories from both groups and remove duplicates
combined_root_ids = list(set(top_root_ids_0).union(set(top_root_ids_1)))

# Sort combined_root_ids based on the addicted group's average interactions in descending order
combined_root_ids = df_result_1.loc[combined_root_ids].sort_values(by='divided', ascending=False).index
print(combined_root_ids)


# combined_root_ids = df_combb_root.groupby(['root_name'])['num_interactions'].agg(['sum', 'count']).sort_values(by='sum',ascending=False).iloc[:5].index

# Extract the average interaction values for the combined categories
non_addicted_values = df_result_0.loc[combined_root_ids]['divided'].values
addicted_values = df_result_1.loc[combined_root_ids]['divided'].values

# Plotting
bar_width = 0.35
index = np.arange(len(combined_root_ids))

fig, ax = plt.subplots(figsize=(10, 4))

# Plot addicted users' data
rects1 = ax.bar(index - bar_width / 2, addicted_values, bar_width, label='Addicted',alpha=0.75)

# Plot non-addicted users' data
rects2 = ax.bar(index + bar_width / 2, non_addicted_values, bar_width, label='Non-Addicted',alpha=0.75)

# Add labels and title
ax.set_xlabel('Video Category', fontsize=21)
ax.set_ylabel('Average Interactions', fontsize=21)
ax.set_ylim(0,27)

# ax.set_xticks(index,labels=['Films and Shorts','Gaming','News','Daily Life','Food'],fontsize=18)
ax.set_xticks(index,labels=['Films and Shorts','Gaming','News','Daily Life','Food'],fontsize=19)

# Add legend
legend = plt.legend(fontsize=18)
# ax.spines[['right', 'top']].set_visible(False)

# Adjust layout and show plot
plt.tight_layout()
plt.savefig('./addiction1_xgboost_20241201revision_output/figs/average_interactions_by_category_soft.pdf', format='pdf', dpi=1000, bbox_inches='tight')
plt.show()

In [ ]:
df_monthly_counts_soft = df_combined_all.groupby(['month', 'preds_soft']).size().reset_index(name='count')

fig, ax = plt.subplots(figsize=(7, 4))
sns.lineplot(data=df_monthly_counts_soft, x='month', y='count', hue='preds_soft', hue_order=[1, 0], 
             linewidth=3, palette={0: "#FC8D62", 1: "#8DA0CB"}, marker="o")

# Add labels and title
plt.xlabel('Month', fontsize=20)
plt.ylabel('User Counts', fontsize=20)

# Create custom legend with thicker lines
legend_handles = [Line2D([0], [0], color="#8DA0CB", lw=3, label='Addicted'),
                  Line2D([0], [0], color="#FC8D62", lw=3, label='Non-Addicted')]

plt.legend(handles=legend_handles, fontsize=18)

# Add grid and format layout
plt.grid(True, alpha=0.4)
plt.tight_layout()

# Save plot
plt.savefig('./addiction1_xgboost_20241201revision_output/figs/temporal_user_counts_soft.pdf', format='pdf', dpi=1000, bbox_inches='tight')

# Show plot
plt.show()


# For preds_3_label_criteriaGood
df_monthly_counts_3labels = df_combined_all.groupby(['month', 'preds_3_label_criteria']).size().reset_index(name='count')

fig, ax = plt.subplots(figsize=(7, 4))

# Use lineplot to show the user counts across months for each addiction level
sns.lineplot(data=df_monthly_counts_3labels, x='month', y='count', hue='preds_3_label_criteria', hue_order=[2, 1, 0], 
             linewidth=3, palette={0: "#FEB29B", 1: "#8DA0CB", 2: "#9467BD"}, marker="o")

# Add labels and title
plt.xlabel('Month', fontsize=20)
plt.ylabel('User Counts', fontsize=20)

# Create custom legend with thicker lines
legend_handles = [Line2D([0], [0], color="#9467BD", lw=3, label='Severely Addicted'),
                  Line2D([0], [0], color="#8DA0CB", lw=3, label='Mildly Addicted'),
                  Line2D([0], [0], color="#FEB29B", lw=3, label='Non-Addicted')]

plt.legend(handles=legend_handles, fontsize=18)

# Add grid and format layout
plt.grid(True, alpha=0.4)
plt.tight_layout()

# Save plot
# plt.savefig('./addiction1_xgboost_20241201revision_output/figs/temporal_user_counts_3labels.pdf', format='pdf', dpi=1000, bbox_inches='tight')

# Show plot
plt.show()

In [ ]:
folder_to_zip = './addiction1_xgboost_20241201revision_output/figs'
output_filename = './addiction1_xgboost_20241201revision_output/figs_zip5'

# Create a zip archive
shutil.make_archive(output_filename, 'zip', folder_to_zip)